In [12]:
import pandas as pd
import os

def format_siren(siren):
    """Formate les SIREN pour avoir exactement 9 caractères en ajoutant des zéros au début si nécessaire"""
    if pd.isna(siren):
        return siren
    
    siren_str = str(siren).strip()
    
    # Si le SIREN a moins de 9 caractères, ajouter des zéros au début
    if len(siren_str) < 9:
        siren_str = siren_str.zfill(9)  # zfill ajoute des zéros au début jusqu'à atteindre la longueur spécifiée
    
    return siren_str

def main():
    # Chemin du fichier Excel
    excel_file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//20250424//extract_ligne_2069A 2022.xlsx"
    
    # Chemin du fichier Parquet
    parquet_file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//cir_millesime_2022_ss_dbls.parquet"
    
    # Vérification supplémentaire du chemin (debug)
    print(f"Chemin absolu attendu pour le fichier Excel : {os.path.abspath(excel_file_path)}")
    print(f"Le fichier Excel existe ? {os.path.exists(excel_file_path)}")
    print(f"Chemin absolu attendu pour le fichier Parquet : {os.path.abspath(parquet_file_path)}")
    print(f"Le fichier Parquet existe ? {os.path.exists(parquet_file_path)}")    
    
    # Vérifier si les fichiers existent
    if not os.path.exists(excel_file_path):
        print(f"Erreur: Le fichier Excel {excel_file_path} n'existe pas.")
        return
    
    if not os.path.exists(parquet_file_path):
        print(f"Erreur: Le fichier Parquet {parquet_file_path} n'existe pas.")
        return
    
    try:
        # Charger les données Excel, avec le nom de feuille corrigé
        sheet_name = "liste SIREN-dépenses"
        print(f"Chargement du fichier Excel {excel_file_path}, feuille '{sheet_name}'...")
        
        try:
            excel_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
        except ValueError as e:
            print(f"Erreur lors du chargement de la feuille '{sheet_name}': {e}")
            # Liste toutes les feuilles disponibles
            xls = pd.ExcelFile(excel_file_path)
            print(f"Feuilles disponibles dans le fichier: {xls.sheet_names}")
            
            # Essayer avec un nom similaire si disponible
            alternative_names = [name for name in xls.sheet_names if 'siren' in name.lower()]
            if alternative_names:
                sheet_name = alternative_names[0]
                print(f"Tentative avec la feuille alternative: '{sheet_name}'")
                excel_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
            else:
                print("Aucune feuille alternative trouvée. Veuillez vérifier le nom exact de la feuille.")
                return
        
        # Vérifier que la colonne SIREN existe
        if 'SIREN' not in excel_df.columns:
            print("Erreur: La colonne 'SIREN' n'existe pas dans la feuille.")
            print(f"Colonnes disponibles: {excel_df.columns.tolist()}")
            
            # Chercher des colonnes similaires
            potential_siren_cols = [col for col in excel_df.columns if 'siren' in col.lower()]
            if potential_siren_cols:
                siren_col_excel = potential_siren_cols[0]
                print(f"Utilisation de la colonne alternative '{siren_col_excel}' comme SIREN")
                excel_df['SIREN'] = excel_df[siren_col_excel]
            else:
                return
        
        # Formatage des SIRENs dans le fichier Excel
        print("Formatage des SIRENs du fichier Excel...")
        excel_df['SIREN'] = excel_df['SIREN'].apply(format_siren)
        
        # Nombre total de SIRENs dans le fichier Excel
        total_sirens_excel = len(excel_df['SIREN'].unique())
        print(f"Nombre total de SIRENs uniques dans le fichier Excel: {total_sirens_excel}")
        
        # Charger les données Parquet
        print(f"Chargement du fichier Parquet {parquet_file_path}...")
        try:
            parquet_df = pd.read_parquet(parquet_file_path)
        except Exception as e:
            print(f"Erreur lors du chargement du fichier Parquet: {e}")
            print("Vérifiez que le package pyarrow ou fastparquet est installé.")
            return
        
        # Vérifier la colonne SIREN dans le fichier Parquet
        siren_col = None
        potential_siren_cols = ['SIREN', 'SIREN_DECLARANT', 'siren', 'siren_declarant']
        
        for col in potential_siren_cols:
            if col in parquet_df.columns:
                siren_col = col
                break
        
        if siren_col is None:
            print("Erreur: Aucune colonne SIREN n'a été trouvée dans le fichier Parquet.")
            # Afficher les colonnes disponibles
            print(f"Colonnes disponibles dans le fichier Parquet: {parquet_df.columns.tolist()}")
            return
        
        # Formatage des SIRENs dans le fichier Parquet
        print(f"Formatage des SIRENs du fichier Parquet (colonne '{siren_col}')...")
        parquet_df[siren_col] = parquet_df[siren_col].apply(format_siren)
        
        # Convertir en ensembles pour une recherche plus rapide
        sirens_excel = set(excel_df['SIREN'].unique())
        sirens_parquet = set(parquet_df[siren_col].unique())
        
        # Vérifier quels SIRENs de l'Excel sont absents du Parquet
        sirens_missing = sirens_excel - sirens_parquet
        
        # Résultats
        if len(sirens_missing) == 0:
            print("✓ Tous les SIRENs du fichier Excel sont présents dans le fichier Parquet.")
        else:
            print(f"⚠ {len(sirens_missing)} SIRENs du fichier Excel sont absents du fichier Parquet:")
            missing_df = excel_df[excel_df['SIREN'].isin(sirens_missing)].copy()
            
            # Afficher les 10 premiers SIRENs manquants avec leurs montants RD si disponible
            montant_col = 'montant RD'
            if montant_col in missing_df.columns:
                print(missing_df[['SIREN', montant_col]].head(10))
            else:
                # Chercher une colonne qui pourrait contenir les montants
                montant_cols = [col for col in missing_df.columns if 'montant' in col.lower() or 'rd' in col.lower()]
                if montant_cols:
                    print(missing_df[['SIREN', montant_cols[0]]].head(10))
                else:
                    print(missing_df['SIREN'].head(10).tolist())
            
            # Sauvegarder les SIRENs manquants dans un fichier CSV
            output_file = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//sirens_manquants.csv"
            missing_df.to_csv(output_file, index=False)
            print(f"La liste complète des SIRENs manquants a été sauvegardée dans {output_file}")
        
        # Statistiques supplémentaires
        print("\nStatistiques:")
        print(f"- Nombre total de SIRENs dans le fichier Excel: {total_sirens_excel}")
        print(f"- Nombre total de SIRENs dans le fichier Parquet: {len(sirens_parquet)}")
        print(f"- Nombre de SIRENs de l'Excel présents dans le Parquet: {total_sirens_excel - len(sirens_missing)}")
        print(f"- Pourcentage de couverture: {((total_sirens_excel - len(sirens_missing)) / total_sirens_excel) * 100:.2f}%")
        
    except Exception as e:
        import traceback
        print(f"Une erreur s'est produite: {e}")
        print(traceback.format_exc())

if __name__ == "__main__":
    main()

Chemin absolu attendu pour le fichier Excel : M:\str-dgri-gecir-donnees-fiscales\x-pour MF-SAMB\20250424\extract_ligne_2069A 2022.xlsx
Le fichier Excel existe ? True
Chemin absolu attendu pour le fichier Parquet : M:\str-dgri-gecir-donnees-fiscales\x-pour MF-SAMB\output\output_parquet\cir_millesime_2022_ss_dbls.parquet
Le fichier Parquet existe ? True
Chargement du fichier Excel M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//20250424//extract_ligne_2069A 2022.xlsx, feuille 'liste SIREN-dépenses'...
Formatage des SIRENs du fichier Excel...
Nombre total de SIRENs uniques dans le fichier Excel: 947
Chargement du fichier Parquet M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//cir_millesime_2022_ss_dbls.parquet...
Formatage des SIRENs du fichier Parquet (colonne 'siren_declarant')...
⚠ 68 SIRENs du fichier Excel sont absents du fichier Parquet:
        SIREN  montant RD
7   319772745     37664.0
12  325706422   2921064.0
15  328170550     19382.0
16  32889

# Recalcule Creance pour toute les entreprises Comparaison Calcule vs Non Calcule (2022)


In [5]:
df = pd.read_parquet("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//cir_millesime_2022_ss_dbls.parquet")
df.to_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//cir_millesime_2022_ss_dbls.csv", index=False)

In [30]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Charger les données
df = pd.read_parquet("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//cir_millesime_2022_ss_dbls.parquet")

def convertir_en_nombre(valeur, defaut=0):
    """Convertit une valeur en nombre de façon sécurisée"""
    if pd.isna(valeur) or valeur == '' or valeur is None:
        return defaut
    try:
        return float(valeur)
    except:
        return defaut

def comparer_cir_declare_recalcule():
    """Compare le CIR déclaré et le CIR recalculé"""
    print(f"Nombre total d'entreprises: {len(df):,}")

    # Colonnes nécessaires pour recalculer le CIR
    colonnes = [
        'siren_declarant', 'siren_deposant', 'DESIGN', 'COMPLT_DESIGN',
        'MT_NET_DEP_RD', 'MT_NET_DEP_RD_DOM', 'MT_TOT_CIR_CI_COLL_CII',
        'MT_CIR_RECH_YC_QP_MOINS_DE100', 'MT_CI_COLL_APRS_MINIMI', 'MT_CII_YC_QP', 'MTotal_CRC',
        'DOT_AMORT_IMMO', 'DOT_AMORT_IMMO_SINISTR', 'DEP_CHERCH_TECH', 'REM_SAL_INV',
        'DEP_JD', 'OTR_DEP_FONCT', 'FRAIS_BREV_COV', 'DEP_MAINT_BREV_COV',
        'DOT_AMORT_BREV', 'DEP_NORMALI', 'PRIM_COTIZ', 'DEP_VEIL_TECHNO',
        'MT_DEP_FONCT_TOT', 'MT_TOT_RD_1',
        'DEP_EXT_LIE_FR', 'DEP_EXT_LIE_ETR', 'DEP_EXT_NON_LIE_FR', 'DEP_EXT_NON_LIE_ETR',
        'MT_TOT_DEP_EXT_ORG_AGREE', 'PLAF_OP_EXT', 'MT_TOT_OP_SOUS_TRAIT',
        'PLAF_OP_EXT_ORG_AGRE_LIE', 'PLAF_OP_EXT_ORG_AGRE_NON_LIE', 'PLAF_GNRL_DEP_EXT', 'MT_DEP_EXT_PLAF',
        'MT_TOT_RD_2',
        'MT_AID_SUBV', 'MT_ENC_PRESTA', 'MT_DEP_CONSEILS_CIR', 'REMBST_SUBV',
        'FRAIS_COLL', 'FRAIS_DEF_DESSIN', 'MT_TOT_DEP_COLL', 'MT_AID_SUBV_COLL',
        'MT_DEP_CONSEILS_CIR_COLL', 'REMBST_SUBV_COLL', 'MT_NET_DEP_COLL', 'MT_NET_DEP_COLL_DOM',
        'DOT_AMORT_IMMO_INO', 'DEP_PERSONEL_INO', 'OTR_DEP_FONCT_INO', 'FRAIS_BREV_COV_INO',
        'FRAIS_DEF_BREV_INO', 'OP_INOV_EXT', 'MT_TOT_DEP_INO', 'MT_TOT_DEP_INO_PLAF',
        'MT_AID_SUBV_INO', 'MT_ENC_PRESTA_INO', 'MT_DEP_CONSEILS_CII', 'REMBST_SUBV_INO',
        'MT_NET_DEP_INO', 'MT_NET_DEP_INO_DOM', 'MT_NET_DEP_INO_MPE_CORSE', 'MT_NET_DEP_INO_ME_CORSE',
        'DEP_CRC', 'DEP_CRC_FR', 'DEP_CRC_ETR', 'DEP_CRC_PLAF', 'AIDE_PUB_CRC',
        'AIDE_PUB_REMB_CRC', 'MT_NET_DEP_CRC', 'MT_NET_DEP_CRC_PME', 'MT_CRC','MT_QP_COLL_RECU_MOINS_DE100',
        'MT_QP_CIR_RECU_BIS','MT_QP_COLL_RECU_BIS','MT_AIDE_MINIMI_COLL','MT_QP_CII','QP_MT_CRC','MT_CII_CORSE',
        'MT_CI_COLL_APRS_MINIMI_DOM', 'MT_CI_COLL_APRES_MINIMI_PLUS_DE100_DOM', 'MT_QP_CIR_RECU', 'MT_AIDE_MINIMI_MOINS_DE100'
    ]

    # Convertir en numérique et créer une copie défragmentée
    df_tmp = df.copy()
    for col in colonnes:
        if col in df_tmp.columns:
            if col not in ['siren_declarant', 'siren_deposant', 'DESIGN', 'COMPLT_DESIGN']:
                df_tmp[col] = df_tmp[col].apply(convertir_en_nombre)
        else:
            if col not in ['siren_declarant', 'siren_deposant', 'DESIGN', 'COMPLT_DESIGN']:
                df_tmp[col] = 0

    # Créer un dictionnaire pour stocker toutes les colonnes calculées
    # et les ajouter en une seule fois à la fin pour éviter la fragmentation (afin d'éviter d'augmenter la complexité du code)
    calc_columns = {}

    ## I - DÉPENSES DE RECHERCHE (CIR-RECHERCHE)

    # 1. Dépenses internes (Section I-A)

    # Ligne 6: Autres dépenses de fonctionnement
    calc_columns['LIGNE_6_CALC'] = (df_tmp['DOT_AMORT_IMMO'] * 0.75) + \
                               ((df_tmp['DEP_CHERCH_TECH'] + df_tmp['REM_SAL_INV']) * 0.43) + \
                               df_tmp['DEP_JD']

    # Ligne 7: Total dépenses de fonctionnement
    calc_columns['LIGNE_7_CALC'] = df_tmp['DOT_AMORT_IMMO'] + df_tmp['DOT_AMORT_IMMO_SINISTR'] + \
                               df_tmp['DEP_CHERCH_TECH'] + df_tmp['REM_SAL_INV'] + \
                               df_tmp['DEP_JD'] + calc_columns['LIGNE_6_CALC']


    # Appliquer les plafonds pour les lignes concernées
    # Ligne 11: Dépenses liées à la normalisation , utiliser directement la valeur renseignée
    calc_columns['DEP_NORMALI_RECALC'] = df_tmp['DEP_NORMALI']  # Conserver la valeur telle quelle

    # Ligne 12: Primes et cotisations (plafond 60 000 €)
    calc_columns['PRIM_COTIZ_PLAFONNEES'] = np.minimum(df_tmp['PRIM_COTIZ'], 60000)

    # Ligne 13: Dépenses de veille technologique (plafond 60 000 €)
    calc_columns['DEP_VEIL_TECHNO_PLAFONNEES'] = np.minimum(df_tmp['DEP_VEIL_TECHNO'], 60000)


    # Ligne 14: Total dépenses internes avec plafonds appliqués
    calc_columns['LIGNE_14_CALC'] = calc_columns['LIGNE_7_CALC'] + df_tmp['FRAIS_BREV_COV'] + \
                                df_tmp['DEP_MAINT_BREV_COV'] + df_tmp['DOT_AMORT_BREV'] + \
                                calc_columns['DEP_NORMALI_RECALC'] + calc_columns['PRIM_COTIZ_PLAFONNEES'] + \
                                calc_columns['DEP_VEIL_TECHNO_PLAFONNEES']

    # 2. Dépenses externalisées (Section I-B)

    # Ligne 17: Total dépenses externalisées
    calc_columns['LIGNE_17_CALC'] = df_tmp['DEP_EXT_LIE_FR'] + df_tmp['DEP_EXT_LIE_ETR'] + \
                                df_tmp['DEP_EXT_NON_LIE_FR'] + df_tmp['DEP_EXT_NON_LIE_ETR']

    # Préparation pour les calculs complexes nécessitant des références à d'autres colonnes calculées
    # Ajoutons temporairement les colonnes calculées au DataFrame
    df_calc = pd.DataFrame(calc_columns)
    df_calc_temp = pd.concat([df_tmp, df_calc], axis=1)

    # Appliquer les plafonnements
    # Ligne 18: Plafond global (3x dépenses internes)
    calc_columns['LIGNE_18_CALC'] = np.minimum(df_calc_temp['LIGNE_17_CALC'], df_calc_temp['LIGNE_14_CALC'] * 3)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_18_CALC'] = calc_columns['LIGNE_18_CALC']

    # Pour les calculs complexes, utiliser des boucles au lieu de apply
    # Ligne 19: Plafond pour organismes liés (2M€)
    ligne_19_values = []
    for _, row in df_calc_temp.iterrows():
        dep_liees = row['DEP_EXT_LIE_FR'] + row['DEP_EXT_LIE_ETR']
        plafond = min(row['LIGNE_18_CALC'], 2000000)
        ligne_19_values.append(min(dep_liees, plafond))
    calc_columns['LIGNE_19_CALC'] = np.array(ligne_19_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_19_CALC'] = calc_columns['LIGNE_19_CALC']

    # Ligne 20: Plafond pour organismes non liés (10M€)
    ligne_20_values = []
    for _, row in df_calc_temp.iterrows():
        dep_non_liees = row['DEP_EXT_NON_LIE_FR'] + row['DEP_EXT_NON_LIE_ETR']
        plafond_restant = min(row['LIGNE_18_CALC'] - row['LIGNE_19_CALC'], 10000000)
        ligne_20_values.append(min(dep_non_liees, plafond_restant))
    calc_columns['LIGNE_20_CALC'] = np.array(ligne_20_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_20_CALC'] = calc_columns['LIGNE_20_CALC']

    # Ligne 21: Total après plafonnements
    ligne_21_values = []
    for _, row in df_calc_temp.iterrows():
        ligne_21_values.append(min(row['LIGNE_19_CALC'] + row['LIGNE_20_CALC'], 10000000))
    calc_columns['LIGNE_21_CALC'] = np.array(ligne_21_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_21_CALC'] = calc_columns['LIGNE_21_CALC']

    # 3. Montant total et net (Section I-C)

    # Ligne 22: Total dépenses
    calc_columns['LIGNE_22_CALC'] = df_calc_temp['LIGNE_14_CALC'] + df_calc_temp['LIGNE_21_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_22_CALC'] = calc_columns['LIGNE_22_CALC']

    # Ligne 26a: Montant net
    calc_columns['LIGNE_26A_CALC'] = df_calc_temp['LIGNE_22_CALC'] - df_tmp['MT_AID_SUBV'] - \
                                  df_tmp['MT_ENC_PRESTA'] - df_tmp['MT_DEP_CONSEILS_CIR'] + \
                                  df_tmp['REMBST_SUBV']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_26A_CALC'] = calc_columns['LIGNE_26A_CALC']

    # Ligne 26b: Utiliser la valeur de MT_NET_DEP_RD_DOM directement
    calc_columns['LIGNE_26B_CALC'] = df_tmp['MT_NET_DEP_RD_DOM']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_26B_CALC'] = calc_columns['LIGNE_26B_CALC']

    ## II - DÉPENSES DE COLLECTION (CIR-COLLECTION)

    # Ligne 28: Frais de défense des dessins et modèles (plafond 60 000 €)
    calc_columns['FRAIS_DEF_DESSIN_PLAFONNES'] = np.minimum(df_tmp['FRAIS_DEF_DESSIN'], 60000)

    # Ligne 29: Total des dépenses de collection
    calc_columns['LIGNE_29_CALC'] = df_tmp['FRAIS_COLL'] + calc_columns['FRAIS_DEF_DESSIN_PLAFONNES']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_29_CALC'] = calc_columns['LIGNE_29_CALC']

    # Ligne 33a: Montant net des dépenses de collection
    calc_columns['LIGNE_33A_CALC'] = df_calc_temp['LIGNE_29_CALC'] - df_tmp['MT_AID_SUBV_COLL'] - \
                                  df_tmp['MT_DEP_CONSEILS_CIR_COLL'] + df_tmp['REMBST_SUBV_COLL']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_33A_CALC'] = calc_columns['LIGNE_33A_CALC']

    # Ligne 33b: Utiliser la valeur de MT_NET_DEP_COLL_DOM directement
    calc_columns['LIGNE_33B_CALC'] = df_tmp['MT_NET_DEP_COLL_DOM']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_33B_CALC'] = calc_columns['LIGNE_33B_CALC']

    # Ligne 34a: Montant net total des dépenses de recherche et de collection
    calc_columns['LIGNE_34A_CALC'] = df_calc_temp['LIGNE_26A_CALC'] + df_calc_temp['LIGNE_33A_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_34A_CALC'] = calc_columns['LIGNE_34A_CALC']

    # Ligne 34b: Montant net total des dépenses de recherche et de collection DOM
    calc_columns['LIGNE_34B_CALC'] = df_calc_temp['LIGNE_26B_CALC'] + df_calc_temp['LIGNE_33B_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_34B_CALC'] = calc_columns['LIGNE_34B_CALC']

    ## V - DÉPENSES DE RECHERCHE COLLABORATIVE (CRC)

    # Vérifier si données disponibles pour la recherche collaborative
    if all(col in df_tmp.columns for col in ['DEP_CRC', 'DEP_CRC_FR', 'DEP_CRC_ETR']):
        # Dépenses éligibles
        calc_columns['LIGNE_83_CALC'] = df_tmp['DEP_CRC_FR'] + df_tmp['DEP_CRC_ETR']

        # Plafonnement à 6 000 000 €
        calc_columns['LIGNE_84_CALC'] = np.minimum(calc_columns['LIGNE_83_CALC'], 6000000)

        # Montant net des dépenses de recherche collaborative
        calc_columns['MT_NET_DEP_CRC_CALC'] = calc_columns['LIGNE_84_CALC'] - df_tmp['AIDE_PUB_CRC'] + \
                                          df_tmp['AIDE_PUB_REMB_CRC']

        # CRC selon les taux
        calc_columns['LIGNE_89_CALC'] = ((calc_columns['MT_NET_DEP_CRC_CALC'] - df_tmp['MT_NET_DEP_CRC_PME']) * 0.4) + \
                                     (df_tmp['MT_NET_DEP_CRC_PME'] * 0.5)

        calc_columns['CIR_COLLAB_RECALCULE'] = calc_columns['LIGNE_89_CALC']
        calc_columns['LIGNE_87'] = calc_columns['MT_NET_DEP_CRC_CALC']
    else:
        # Si données non disponibles, mettre à 0
        calc_columns['CIR_COLLAB_RECALCULE'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_87'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_89_CALC'] = np.zeros(len(df_tmp))

    # Mise à jour du DataFrame temporaire
    for col in ['CIR_COLLAB_RECALCULE', 'LIGNE_87', 'LIGNE_89_CALC']:
        if col in calc_columns:
            df_calc_temp[col] = calc_columns[col]

    ## III - CALCUL DU CRÉDIT D'IMPÔT AU TITRE DES DÉPENSES DE RECHERCHE ET DE COLLECTION

    # Identifier les entreprises <= 100M€ en utilisant la somme ligne 34a + ligne 87
    calc_columns['DEPENSES_MOINS_100M'] = (df_calc_temp['LIGNE_34A_CALC'] + df_calc_temp['LIGNE_87']) <= 100000000

    # Mise à jour du DataFrame temporaire
    df_calc_temp['DEPENSES_MOINS_100M'] = calc_columns['DEPENSES_MOINS_100M']

    # A. Dépenses <= 100 000 000 €

    # CIR Recherche pour entreprises <= 100M€
    ligne_36_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_36_values.append((row['LIGNE_26A_CALC'] - row['LIGNE_26B_CALC']) * 0.3 +
                      row['LIGNE_26B_CALC'] * 0.5)
        else:
            ligne_36_values.append(0)
    calc_columns['LIGNE_36_CALC'] = np.array(ligne_36_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_36_CALC'] = calc_columns['LIGNE_36_CALC']

    # Ligne 37: Quote-part de crédit d'impôt résultant de la participation sociétés de personnes
    calc_columns['LIGNE_37'] = df_tmp['MT_QP_CIR_RECU']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_37'] = calc_columns['LIGNE_37']

    # Calcul ligne 38a: Montant du crédit d'impôt pour dépenses de recherche
    calc_columns['LIGNE_38A_CALC'] = df_calc_temp['LIGNE_36_CALC'] + df_calc_temp['LIGNE_37']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_38A_CALC'] = calc_columns['LIGNE_38A_CALC']

    # Calcul ligne 38b: Montant du CIR pour dépenses de recherche DOM
    calc_columns['LIGNE_38B_CALC'] = df_calc_temp['LIGNE_26B_CALC'] * 0.5  # Taux de 50% pour DOM

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_38B_CALC'] = calc_columns['LIGNE_38B_CALC']

    # CIR Collection pour entreprises <= 100M€ (avant plafond de minimis)
    ligne_40_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_40_values.append((row['LIGNE_33A_CALC'] - row['LIGNE_33B_CALC']) * 0.3 +
                      row['LIGNE_33B_CALC'] * 0.5)
        else:
            ligne_40_values.append(0)
    calc_columns['LIGNE_40_CALC'] = np.array(ligne_40_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_40_CALC'] = calc_columns['LIGNE_40_CALC']

    # Ligne 41: Quote-part de crédit d'impôt collection résultant de la participation sociétés de personnes
    # Nous supposons qu'elle est à 0 car elle n'est pas calculable avec les données disponibles
    calc_columns['LIGNE_41'] = df_tmp['MT_QP_COLL_RECU_MOINS_DE100']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_41'] = calc_columns['LIGNE_41']

    # Calcul ligne 42a: Montant total du crédit d'impôt pour dépenses de collection avant plafonnement
    calc_columns['LIGNE_42A_CALC'] = df_calc_temp['LIGNE_40_CALC'] + df_calc_temp['LIGNE_41']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_42A_CALC'] = calc_columns['LIGNE_42A_CALC']

    # Calcul ligne 42b: Montant du crédit collection DOM avant plafonnement
    calc_columns['LIGNE_42B_CALC'] = df_calc_temp['LIGNE_33B_CALC'] * 0.5  # Taux de 50% pour DOM

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_42B_CALC'] = calc_columns['LIGNE_42B_CALC']

    # Ligne 43: Montant des aides de minimis
    calc_columns['LIGNE_43'] = df_tmp['MT_AIDE_MINIMI_MOINS_DE100']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_43'] = calc_columns['LIGNE_43']

    # Calcul ligne 44: Montant cumulé du crédit d'impôt et des aides de minimis
    calc_columns['LIGNE_44'] = df_calc_temp['LIGNE_42A_CALC'] + df_calc_temp['LIGNE_43']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_44'] = calc_columns['LIGNE_44']

    # Calcul ligne 45a: Montant du crédit d'impôt pour dépenses de collection après plafonnement
    ligne_45a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['LIGNE_43'] >= 200000:  # Si aides de minimis déjà à 200k€
            ligne_45a_values.append(0)
        elif row['LIGNE_44'] < 200000:  # Si cumul < 200k€
            ligne_45a_values.append(row['LIGNE_42A_CALC'])
        else:  # Si dépassement
            ligne_45a_values.append(200000 - row['LIGNE_43'])
    calc_columns['LIGNE_45A_CALC'] = np.array(ligne_45a_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_45A_CALC'] = calc_columns['LIGNE_45A_CALC']

    # ligne 45b: Montant du crédit collection DOM après plafonnement

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_45B_CALC'] = df_tmp['MT_CI_COLL_APRS_MINIMI_DOM']

    # Calcul ligne 46a: Montant total du crédit d'impôt au titre des dépenses de recherche et de collection
    calc_columns['LIGNE_46A_CALC'] = df_calc_temp['LIGNE_38A_CALC'] + df_calc_temp['LIGNE_45A_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_46A_CALC'] = calc_columns['LIGNE_46A_CALC']

    # Calcul ligne 46b: Montant du crédit d'impôt recherche et collection exposées dans DOM
    calc_columns['LIGNE_46B_CALC'] = df_calc_temp['LIGNE_38B_CALC'] + df_calc_temp['LIGNE_45B_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_46B_CALC'] = calc_columns['LIGNE_46B_CALC']

    # B. Dépenses > 100 000 000 €

    # Identifier les entreprises > 100M€
    calc_columns['DEPENSES_PLUS_100M'] = ~df_calc_temp['DEPENSES_MOINS_100M']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['DEPENSES_PLUS_100M'] = calc_columns['DEPENSES_PLUS_100M']

    # Limiter les dépenses à 100M€ - dépenses recherche collaborative
    ligne_47a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_47a_values.append(min(row['LIGNE_26A_CALC'], 100000000 - row['LIGNE_87']))
        else:
            ligne_47a_values.append(0)
    calc_columns['LIGNE_47A_CALC'] = np.array(ligne_47a_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_47A_CALC'] = calc_columns['LIGNE_47A_CALC']

    # Proportion DOM dans la limite des 100M€ (ligne 47b)
    ligne_47b_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_47b_values.append(min(row['LIGNE_26B_CALC'], 100000000 - row['LIGNE_87']))
        else:
            ligne_47b_values.append(0)
    calc_columns['LIGNE_47B_CALC'] = np.array(ligne_47b_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_47B_CALC'] = calc_columns['LIGNE_47B_CALC']

    # Calcul CIR recherche première tranche (ligne 48)
    ligne_48_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_48_values.append((row['LIGNE_47A_CALC'] - row['LIGNE_47B_CALC']) * 0.3 +
                           row['LIGNE_47B_CALC'] * 0.5)
        else:
            ligne_48_values.append(0)
    calc_columns['LIGNE_48_CALC'] = np.array(ligne_48_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_48_CALC'] = calc_columns['LIGNE_48_CALC']

    # Dépenses > 100M€ (ligne 49)
    ligne_49_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_49_values.append(max(0, row['LIGNE_26A_CALC'] - (100000000 - row['LIGNE_87'])))
        else:
            ligne_49_values.append(0)
    calc_columns['LIGNE_49_CALC'] = np.array(ligne_49_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_49_CALC'] = calc_columns['LIGNE_49_CALC']

    # CIR recherche deuxième tranche (au-delà de 100M€) (ligne 50)
    calc_columns['LIGNE_50_CALC'] = df_calc_temp['LIGNE_49_CALC'] * 0.05

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_50_CALC'] = calc_columns['LIGNE_50_CALC']

    # CIR recherche total pour entreprises > 100M€ (ligne 51)
    calc_columns['LIGNE_51_CALC'] = df_calc_temp['LIGNE_48_CALC'] + df_calc_temp['LIGNE_50_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_51_CALC'] = calc_columns['LIGNE_51_CALC']

    # Ligne 52: Quote-part de crédit d'impôt recherche résultant de la participation sociétés de personnes
    calc_columns['LIGNE_52'] = df_tmp['MT_QP_CIR_RECU_BIS']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_52'] = calc_columns['LIGNE_52']

    # Calcul ligne 53a: Montant du crédit d'impôt pour dépenses de recherche >100M€
    calc_columns['LIGNE_53A_CALC'] = df_calc_temp['LIGNE_51_CALC'] + df_calc_temp['LIGNE_52']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_53A_CALC'] = calc_columns['LIGNE_53A_CALC']

    # Calcul ligne 53b: Montant du crédit d'impôt pour dépenses de recherche DOM >100M€
    # Comme pour 38b, c'est 50% des dépenses DOM dans la limite du plafond
    calc_columns['LIGNE_53B_CALC'] = df_calc_temp['LIGNE_47B_CALC'] * 0.5

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_53B_CALC'] = calc_columns['LIGNE_53B_CALC']

    # Calcul CIR collection pour entreprises > 100M€

    # Ligne 54a: Montant net total des dépenses de collection
    calc_columns['LIGNE_54A_CALC'] = df_calc_temp['LIGNE_33A_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_54A_CALC'] = calc_columns['LIGNE_54A_CALC']

    # Ligne 54b: Montant net total des dépenses de collection DOM
    calc_columns['LIGNE_54B_CALC'] = df_calc_temp['LIGNE_33B_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_54B_CALC'] = calc_columns['LIGNE_54B_CALC']

    # Plafond disponible après dépenses recherche (ligne 55)
    ligne_55_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_55_values.append(max(0, 100000000 - row['LIGNE_87'] - row['LIGNE_47A_CALC']))
        else:
            ligne_55_values.append(0)
    calc_columns['LIGNE_55_CALC'] = np.array(ligne_55_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_55_CALC'] = calc_columns['LIGNE_55_CALC']

    # CIR collection première tranche (ligne 56)
    ligne_56_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            dep_coll_plaf = min(row['LIGNE_54A_CALC'], row['LIGNE_55_CALC'])
            dep_coll_dom_plaf = min(row['LIGNE_54B_CALC'], row['LIGNE_55_CALC'])
            ligne_56_values.append((dep_coll_plaf - dep_coll_dom_plaf) * 0.3 + dep_coll_dom_plaf * 0.5)
        else:
            ligne_56_values.append(0)
    calc_columns['LIGNE_56_CALC'] = np.array(ligne_56_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_56_CALC'] = calc_columns['LIGNE_56_CALC']

    # CIR collection deuxième tranche (ligne 57)
    ligne_57_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_57_values.append(max(0, row['LIGNE_54A_CALC'] - row['LIGNE_55_CALC']) * 0.05)
        else:
            ligne_57_values.append(0)
    calc_columns['LIGNE_57_CALC'] = np.array(ligne_57_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_57_CALC'] = calc_columns['LIGNE_57_CALC']

    # CIR collection avant plafonnement de minimis (ligne 58)
    calc_columns['LIGNE_58_CALC'] = df_calc_temp['LIGNE_56_CALC'] + df_calc_temp['LIGNE_57_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_58_CALC'] = calc_columns['LIGNE_58_CALC']

    # Ligne 59: Quote-part de crédit d'impôt collection résultant de la participation sociétés de personnes
    calc_columns['LIGNE_59'] = df_tmp['MT_QP_COLL_RECU_BIS']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_59'] = calc_columns['LIGNE_59']

    # Ligne 60: Montant du crédit d'impôt collection avant plafonnement de minimis
    calc_columns['LIGNE_60'] = df_calc_temp['LIGNE_58_CALC'] + df_calc_temp['LIGNE_59']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_60'] = calc_columns['LIGNE_60']

    # Ligne 61: Montant des aides de minimis
    calc_columns['LIGNE_61'] = df_tmp['MT_AIDE_MINIMI_COLL']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_61'] = calc_columns['LIGNE_61']

    # Ligne 62: Montant cumulé du crédit d'impôt et des aides de minimis
    calc_columns['LIGNE_62'] = df_calc_temp['LIGNE_60'] + df_calc_temp['LIGNE_61']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_62'] = calc_columns['LIGNE_62']

    # Ligne 63a: Montant du crédit d'impôt pour dépenses de collection après plafonnement
    ligne_63a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['LIGNE_61'] >= 200000:  # Si aides de minimis déjà à 200k€
            ligne_63a_values.append(0)
        elif row['LIGNE_62'] < 200000:  # Si cumul < 200k€
            ligne_63a_values.append(row['LIGNE_60'])
        else:  # Si dépassement
            ligne_63a_values.append(200000 - row['LIGNE_61'])
    calc_columns['LIGNE_63A_CALC'] = np.array(ligne_63a_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_63A_CALC'] = calc_columns['LIGNE_63A_CALC']

    # Ligne 63b: Montant du crédit d'impôt pour dépenses de collection DOM après plafonnement

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_63B_CALC'] = df_tmp['MT_CI_COLL_APRES_MINIMI_PLUS_DE100_DOM']

    # Ligne 64a: Montant total du crédit d'impôt au titre des dépenses de recherche et de collection
    calc_columns['LIGNE_64A_CALC'] = df_calc_temp['LIGNE_53A_CALC'] + df_calc_temp['LIGNE_63A_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_64A_CALC'] = calc_columns['LIGNE_64A_CALC']

    # Ligne 64b: Montant du crédit d'impôt recherche et collection exposées dans DOM
    calc_columns['LIGNE_64B_CALC'] = df_calc_temp['LIGNE_53B_CALC'] + df_calc_temp['LIGNE_63B_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_64B_CALC'] = calc_columns['LIGNE_64B_CALC']

    # Somme du CIR recherche selon le cas (<=100M€ ou >100M€)
    cir_recherche_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            cir_recherche_values.append(row['LIGNE_38A_CALC'])
        else:
            cir_recherche_values.append(row['LIGNE_53A_CALC'])
    calc_columns['CIR_RECHERCHE_RECALCULE'] = np.array(cir_recherche_values)

    # Somme du CIR collection après plafonnement de minimis selon le cas
    cir_collection_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            cir_collection_values.append(row['LIGNE_45A_CALC'])
        else:
            cir_collection_values.append(row['LIGNE_63A_CALC'])
    calc_columns['CIR_COLLECTION_RECALCULE'] = np.array(cir_collection_values)

    # Mise à jour du DataFrame temporaire
    for col in ['CIR_RECHERCHE_RECALCULE', 'CIR_COLLECTION_RECALCULE']:
        df_calc_temp[col] = calc_columns[col]

    ## IV - DÉPENSES D'INNOVATION (CIR-INNOVATION)

    # Vérifier si les données d'innovation sont disponibles
    if all(col in df_tmp.columns for col in ['DOT_AMORT_IMMO_INO', 'DEP_PERSONEL_INO']):
        # Autres dépenses de fonctionnement
        calc_columns['LIGNE_67_CALC'] = (df_tmp['DOT_AMORT_IMMO_INO'] * 0.75) + \
                                     (df_tmp['DEP_PERSONEL_INO'] * 0.43)

        # Total dépenses d'innovation
        calc_columns['LIGNE_71_CALC'] = df_tmp['DOT_AMORT_IMMO_INO'] + df_tmp['DEP_PERSONEL_INO'] + \
                                     calc_columns['LIGNE_67_CALC'] + df_tmp['FRAIS_BREV_COV_INO'] + \
                                     df_tmp['FRAIS_DEF_BREV_INO'] + df_tmp['OP_INOV_EXT']

        # Plafonnement à 400 000 €
        calc_columns['LIGNE_72_CALC'] = np.minimum(calc_columns['LIGNE_71_CALC'], 400000)

        # Montant net des dépenses d'innovation
        calc_columns['LIGNE_77A_CALC'] = calc_columns['LIGNE_72_CALC'] - df_tmp['MT_AID_SUBV_INO'] - \
                                       df_tmp['MT_ENC_PRESTA_INO'] - df_tmp['MT_DEP_CONSEILS_CII'] + \
                                       df_tmp['REMBST_SUBV_INO']

        # Calcul des parts DOM et Corse
        calc_columns['LIGNE_77B_CALC'] = df_tmp['MT_NET_DEP_INO_DOM']
        calc_columns['LIGNE_77C_CALC'] = df_tmp['MT_NET_DEP_INO_MPE_CORSE']
        calc_columns['LIGNE_77D_CALC'] = df_tmp['MT_NET_DEP_INO_ME_CORSE']

        # Mise à jour du DataFrame temporaire
        for col in ['LIGNE_67_CALC', 'LIGNE_71_CALC', 'LIGNE_72_CALC', 'LIGNE_77A_CALC',
                    'LIGNE_77B_CALC', 'LIGNE_77C_CALC', 'LIGNE_77D_CALC']:
            df_calc_temp[col] = calc_columns[col]

        # CIR Innovation selon les taux
        calc_columns['LIGNE_78_CALC'] = ((df_calc_temp['LIGNE_77A_CALC'] - df_calc_temp['LIGNE_77B_CALC'] -
                                      df_calc_temp['LIGNE_77C_CALC'] - df_calc_temp['LIGNE_77D_CALC']) * 0.2) + \
                                     (df_calc_temp['LIGNE_77B_CALC'] * 0.4) + \
                                     (df_calc_temp['LIGNE_77C_CALC'] * 0.4) + \
                                     (df_calc_temp['LIGNE_77D_CALC'] * 0.35)

        # Ligne 79: Quote-part de crédit d'impôt innovation résultant de la participation sociétés de personnes
        calc_columns['LIGNE_79'] = df_tmp['MT_QP_CII']

        # Ligne 80a: Montant total du crédit d'impôt au titre des dépenses d'innovation
        calc_columns['LIGNE_80A_CALC'] = calc_columns['LIGNE_78_CALC'] + calc_columns['LIGNE_79']

        # Ligne 80b: Montant du crédit d'impôt pour dépenses d'innovation DOM
        calc_columns['LIGNE_80B_CALC'] = df_tmp['MT_CII_YC_QP_DOM']

        # Ligne 80c: Montant du crédit d'impôt pour dépenses d'innovation Corse
        calc_columns['LIGNE_80C_CALC'] = df_tmp['MT_CII_CORSE']

        calc_columns['CIR_INNOVATION_RECALCULE'] = calc_columns['LIGNE_80A_CALC']
    else:
        # Si données non disponibles, mettre à 0 (afin d'éviter des erreurs)
        calc_columns['CIR_INNOVATION_RECALCULE'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_78_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_79'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_80A_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_80B_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_80C_CALC'] = np.zeros(len(df_tmp))

    # Mise à jour du DataFrame temporaire
    for col in ['CIR_INNOVATION_RECALCULE', 'LIGNE_78_CALC', 'LIGNE_79', 'LIGNE_80A_CALC',
               'LIGNE_80B_CALC', 'LIGNE_80C_CALC']:
        if col in calc_columns:
            df_calc_temp[col] = calc_columns[col]

    # Ligne 81a: Montant total du crédit d'impôt au titre des dépenses de recherche, de collection et d'innovation
    ligne_81a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_81a_values.append(row['LIGNE_46A_CALC'] + row['LIGNE_80A_CALC'])
        else:
            ligne_81a_values.append(row['LIGNE_64A_CALC'] + row['LIGNE_80A_CALC'])
    calc_columns['LIGNE_81A_CALC'] = np.array(ligne_81a_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_81A_CALC'] = calc_columns['LIGNE_81A_CALC']

    # Ligne 81b: Montant du crédit d'impôt recherche, collection et innovation DOM
    ligne_81b_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_81b_values.append(row['LIGNE_46B_CALC'] + row['LIGNE_80B_CALC'])
        else:
            ligne_81b_values.append(row['LIGNE_64B_CALC'] + row['LIGNE_80B_CALC'])
    calc_columns['LIGNE_81B_CALC'] = np.array(ligne_81b_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_81B_CALC'] = calc_columns['LIGNE_81B_CALC']


    # Calcul ligne 90: Quote-part de crédit d'impôt résultant de la participation sociétés de personnes
    calc_columns['LIGNE_90'] = df_tmp['QP_MT_CRC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_90'] = calc_columns['LIGNE_90']

    # Calcul ligne 91: Montant total du crédit d'impôt au titre des dépenses de recherche collaborative
    calc_columns['LIGNE_91_CALC'] = df_calc_temp['LIGNE_89_CALC'] + df_calc_temp['LIGNE_90']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_91_CALC'] = calc_columns['LIGNE_91_CALC']

## ===========================================================
    ## TOTAL DU CRÉDIT D'IMPÔT FINAL (CIR + CII + CRC)
    ## ===========================================================

    # Rappel des composantes à additionner :
    # CIR R&C = CIR_RECHERCHE_RECALCULE + CIR_COLLECTION_RECALCULE (déjà calculés)
    # CII Total = CIR_INNOVATION_RECALCULE (basé sur L80a) (déjà calculé)
    # CRC Total = LIGNE_91_CALC (L89 + L90) (déjà calculé)

    # Initialiser le total avec les composantes Recherche et Collection
    calc_columns['CIR_TOTAL_RECALCULE'] = calc_columns['CIR_RECHERCHE_RECALCULE'] + \
                                          calc_columns['CIR_COLLECTION_RECALCULE']

    # Ajouter la composante Innovation (CII) si elle a été calculée
    # (La variable CIR_INNOVATION_RECALCULE contient déjà L80a = L78 + L79)
    if 'CIR_INNOVATION_RECALCULE' in calc_columns:
        # S'assurer que la valeur n'est pas NaN avant d'ajouter
        calc_columns['CIR_TOTAL_RECALCULE'] += np.nan_to_num(calc_columns['CIR_INNOVATION_RECALCULE'])

    # Ajouter la composante Collaborative (CRC) TOTALE (Ligne 91) si elle a été calculée
    # *** C'est la ligne corrigée pour utiliser L91 au lieu de L89 dans la somme ***
    if 'LIGNE_91_CALC' in calc_columns:
        # S'assurer que la valeur n'est pas NaN avant d'ajouter
        calc_columns['CIR_TOTAL_RECALCULE'] += np.nan_to_num(calc_columns['LIGNE_91_CALC'])

    # Optionnel mais recommandé : Mettre à jour df_calc_temp si CIR_TOTAL_RECALCULE est utilisé dans des étapes ultérieures
    # (par exemple pour le calcul de l'écart final)
    # Vérifier si la colonne existe déjà avant d'affecter pour éviter KeyError si non initialisée
    if 'CIR_TOTAL_RECALCULE' not in df_calc_temp.columns:
         df_calc_temp['CIR_TOTAL_RECALCULE'] = 0 # Ou une initialisation appropriée
    df_calc_temp['CIR_TOTAL_RECALCULE'] = calc_columns['CIR_TOTAL_RECALCULE']

    # ANALYSE DES ÉCARTS

    # Écarts sur toutes les lignes recalculées
    # 1. Dépenses internes
    calc_columns['ECART_LIGNE_6'] = calc_columns['LIGNE_6_CALC'] - df_tmp['OTR_DEP_FONCT']
    calc_columns['ECART_LIGNE_7'] = calc_columns['LIGNE_7_CALC'] - df_tmp['MT_DEP_FONCT_TOT']
    calc_columns['ECART_LIGNE_14'] = calc_columns['LIGNE_14_CALC'] - df_tmp['MT_TOT_RD_1']

    # 2. Dépenses externalisées
    calc_columns['ECART_LIGNE_17'] = calc_columns['LIGNE_17_CALC'] - df_tmp['MT_TOT_DEP_EXT_ORG_AGREE']
    calc_columns['ECART_LIGNE_18'] = calc_columns['LIGNE_18_CALC'] - df_tmp['PLAF_OP_EXT']
    calc_columns['ECART_LIGNE_19'] = calc_columns['LIGNE_19_CALC'] - df_tmp['PLAF_OP_EXT_ORG_AGRE_LIE']
    calc_columns['ECART_LIGNE_20'] = calc_columns['LIGNE_20_CALC'] - df_tmp['PLAF_OP_EXT_ORG_AGRE_NON_LIE']
    calc_columns['ECART_LIGNE_21'] = calc_columns['LIGNE_21_CALC'] - df_tmp['MT_DEP_EXT_PLAF']

    # 3. Montant total et net
    calc_columns['ECART_LIGNE_22'] = calc_columns['LIGNE_22_CALC'] - df_tmp['MT_TOT_RD_2']
    calc_columns['ECART_LIGNE_26A'] = calc_columns['LIGNE_26A_CALC'] - df_tmp['MT_NET_DEP_RD']
    calc_columns['ECART_LIGNE_26B'] = calc_columns['LIGNE_26B_CALC'] - df_tmp['MT_NET_DEP_RD_DOM']

    # 4. Dépenses de collection
    calc_columns['ECART_LIGNE_29'] = calc_columns['LIGNE_29_CALC'] - df_tmp['MT_TOT_DEP_COLL']
    calc_columns['ECART_LIGNE_33A'] = calc_columns['LIGNE_33A_CALC'] - df_tmp['MT_NET_DEP_COLL']
    calc_columns['ECART_LIGNE_33B'] = calc_columns['LIGNE_33B_CALC'] - df_tmp['MT_NET_DEP_COLL_DOM']

    # 5. Innovation
    if all(col in df_tmp.columns for col in ['MT_TOT_DEP_INO', 'MT_TOT_DEP_INO_PLAF', 'MT_NET_DEP_INO']):
        calc_columns['ECART_LIGNE_71'] = calc_columns['LIGNE_71_CALC'] - df_tmp['MT_TOT_DEP_INO']
        calc_columns['ECART_LIGNE_72'] = calc_columns['LIGNE_72_CALC'] - df_tmp['MT_TOT_DEP_INO_PLAF']
        calc_columns['ECART_LIGNE_77A'] = calc_columns['LIGNE_77A_CALC'] - df_tmp['MT_NET_DEP_INO']

    # 6. Collaboratif
    if all(col in df_tmp.columns for col in ['DEP_CRC_PLAF', 'MT_NET_DEP_CRC']):
        calc_columns['ECART_LIGNE_84'] = calc_columns['LIGNE_84_CALC'] - df_tmp['DEP_CRC_PLAF']
        calc_columns['ECART_MT_NET_DEP_CRC'] = calc_columns['MT_NET_DEP_CRC_CALC'] - df_tmp['MT_NET_DEP_CRC']

    # 7. Crédits d'impôt
    calc_columns['ECART_CIR_RECHERCHE'] = calc_columns['CIR_RECHERCHE_RECALCULE'] - df_tmp['MT_CIR_RECH_YC_QP_MOINS_DE100']
    calc_columns['ECART_CIR_COLLECTION'] = calc_columns['CIR_COLLECTION_RECALCULE'] - df_tmp['MT_CI_COLL_APRS_MINIMI']

    if 'CIR_INNOVATION_RECALCULE' in df_calc_temp.columns:
        calc_columns['ECART_CIR_INNOVATION'] = df_calc_temp['CIR_INNOVATION_RECALCULE'] - df_tmp['MT_CII_YC_QP']
    else:
        calc_columns['ECART_CIR_INNOVATION'] = -df_tmp['MT_CII_YC_QP']

    if 'CIR_COLLAB_RECALCULE' in df_calc_temp.columns:
        calc_columns['ECART_CIR_COLLAB'] = df_calc_temp['CIR_COLLAB_RECALCULE'] - df_tmp['MT_CRC']
    else:
        calc_columns['ECART_CIR_COLLAB'] = -df_tmp['MT_CRC']

    # Écart total CIR
    calc_columns['ECART_CIR'] = df_calc_temp['CIR_TOTAL_RECALCULE'] - df_tmp['MT_TOT_CIR_CI_COLL_CII']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['ECART_CIR'] = calc_columns['ECART_CIR']

    # Indicateur de correspondance (avec seuil de tolérance de 1)
    correspondance_values = []
    for ecart in calc_columns['ECART_CIR']:
        if abs(ecart) <= 1:
            correspondance_values.append("Oui")
        else:
            correspondance_values.append("Non")
    calc_columns['CORRESPONDANCE'] = np.array(correspondance_values)

    # Écart relatif
    ecart_relatif_values = []
    for i, row in df_calc_temp.iterrows():
        if row['MT_TOT_CIR_CI_COLL_CII'] > 0:
            ecart_relatif_values.append((row['ECART_CIR'] / row['MT_TOT_CIR_CI_COLL_CII'] * 100))
        else:
            ecart_relatif_values.append(100 if row['ECART_CIR'] > 0 else 0)
    calc_columns['ECART_RELATIF'] = np.array(ecart_relatif_values)

    # Création du DataFrame final avec toutes les colonnes calculées
    # (cela évite la fragmentation du DataFrame)
    df_calc_final = pd.DataFrame(calc_columns)

    # Création du DataFrame final pour l'analyse
    df_num = pd.concat([df_tmp, df_calc_final], axis=1)


    ## CRÉATION DU FICHIER DE COMPARAISON DÉTAILLÉ


    # Préparation du dictionnaire de mapping pour la sortie
    # Structure: 'colonne_originale': 'nom_colonne_sortie'
    mapping_colonnes = {
        # Colonnes d'identification
        'siren_declarant': 'SIREN_DECLARANT',
        'siren_deposant': 'SIREN_DEPOSANT',
        'DESIGN': 'DESIGNATION',
        'COMPLT_DESIGN': 'COMPLEMENT_DESIGNATION',

        # I-A. Dépenses internes
        'DOT_AMORT_IMMO': 'L1_DOTATION_AMORT_IMMO',
        'DOT_AMORT_IMMO_SINISTR': 'L2_DOTATION_AMORT_SINISTR',
        'DEP_CHERCH_TECH': 'L3_DEPENSES_PERSONNEL_CHERCHEURS',
        'REM_SAL_INV': 'L4_REMUNERATION_INVENTEURS',
        'DEP_JD': 'L5_DEPENSES_JEUNES_DOCTEURS',
        'OTR_DEP_FONCT': 'L6_AUTRES_DEP_FONCT_DECLARE',
        'LIGNE_6_CALC': 'L6_AUTRES_DEP_FONCT_CALCULE',
        'ECART_LIGNE_6': 'L6_ECART',
        'MT_DEP_FONCT_TOT': 'L7_TOTAL_DEP_FONCT_DECLARE',
        'LIGNE_7_CALC': 'L7_TOTAL_DEP_FONCT_CALCULE',
        'ECART_LIGNE_7': 'L7_ECART',
        'FRAIS_BREV_COV': 'L8_FRAIS_BREVETS_COV',
        'DEP_MAINT_BREV_COV': 'L9_DEPENSES_DEFENSE_BREVETS',
        'DOT_AMORT_BREV': 'L10_DOTATION_AMORT_BREVETS',
        'DEP_NORMALI': 'L11_DEPENSES_NORMALISATION_BRUT',
        'DEP_NORMALI_RECALC': 'L11_DEPENSES_NORMALISATION_DECLARE',
        'PRIM_COTIZ': 'L12_PRIMES_COTISATIONS_BRUT',
        'PRIM_COTIZ_PLAFONNEES': 'L12_PRIMES_COTISATIONS_PLAFONNEES',
        'DEP_VEIL_TECHNO': 'L13_VEILLE_TECHNO_BRUT',
        'DEP_VEIL_TECHNO_PLAFONNEES': 'L13_VEILLE_TECHNO_PLAFONNEE',
        'MT_TOT_RD_1': 'L14_TOTAL_DEPENSES_INTERNES_DECLARE',
        'LIGNE_14_CALC': 'L14_TOTAL_DEPENSES_INTERNES_CALCULE',
        'ECART_LIGNE_14': 'L14_ECART',

        # I-B. Dépenses externalisées
        'DEP_EXT_LIE_FR': 'L15A_DEPENSES_ORG_LIES_FR',
        'DEP_EXT_LIE_ETR': 'L15B_DEPENSES_ORG_LIES_ETR',
        'DEP_EXT_NON_LIE_FR': 'L16A_DEPENSES_ORG_NON_LIES_FR',
        'DEP_EXT_NON_LIE_ETR': 'L16B_DEPENSES_ORG_NON_LIES_ETR',
        'MT_TOT_DEP_EXT_ORG_AGREE': 'L17_TOTAL_DEP_EXTERNALISEES_DECLARE',
        'LIGNE_17_CALC': 'L17_TOTAL_DEP_EXTERNALISEES_CALCULE',
        'ECART_LIGNE_17': 'L17_ECART',
        'PLAF_OP_EXT': 'L18_PLAFOND_GLOBAL_DECLARE',
        'LIGNE_18_CALC': 'L18_PLAFOND_GLOBAL_CALCULE',
        'ECART_LIGNE_18': 'L18_ECART',
        'PLAF_OP_EXT_ORG_AGRE_LIE': 'L19_PLAFOND_ORG_LIES_DECLARE',
        'LIGNE_19_CALC': 'L19_PLAFOND_ORG_LIES_CALCULE',
        'ECART_LIGNE_19': 'L19_ECART',
        'PLAF_OP_EXT_ORG_AGRE_NON_LIE': 'L20_PLAFOND_ORG_NON_LIES_DECLARE',
        'LIGNE_20_CALC': 'L20_PLAFOND_ORG_NON_LIES_CALCULE',
        'ECART_LIGNE_20': 'L20_ECART',
        'MT_DEP_EXT_PLAF': 'L21_TOTAL_DEP_EXT_PLAFONNEES_DECLARE',
        'LIGNE_21_CALC': 'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE',
        'ECART_LIGNE_21': 'L21_ECART',

        # I-C. Montant total et net
        'MT_TOT_RD_2': 'L22_TOTAL_DEPENSES_RECHERCHE_DECLARE',
        'LIGNE_22_CALC': 'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE',
        'ECART_LIGNE_22': 'L22_ECART',
        'MT_AID_SUBV': 'L23A_SUBVENTIONS',
        'MT_ENC_PRESTA': 'L23B_SOMMES_ENCAISSEES_TIERS',
        'MT_DEP_CONSEILS_CIR': 'L24_DEPENSES_CONSEIL_CIR',
        'REMBST_SUBV': 'L25_REMBOURSEMENTS_SUBVENTIONS',
        'MT_NET_DEP_RD': 'L26A_MONTANT_NET_DEPENSES_DECLARE',
        'LIGNE_26A_CALC': 'L26A_MONTANT_NET_DEPENSES_CALCULE',
        'ECART_LIGNE_26A': 'L26A_ECART',
        'MT_NET_DEP_RD_DOM': 'L26B_MONTANT_NET_DEPENSES_DOM_DECLARE',
        'LIGNE_26B_CALC': 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE',
        'ECART_LIGNE_26B': 'L26B_ECART',

        # II. Dépenses de collection
        'FRAIS_COLL': 'L27_FRAIS_COLLECTION',
        'FRAIS_DEF_DESSIN': 'L28_FRAIS_DEFENSE_DESSINS_BRUT',
        'FRAIS_DEF_DESSIN_PLAFONNES': 'L28_FRAIS_DEFENSE_DESSINS_PLAFONNES',
        'MT_TOT_DEP_COLL': 'L29_TOTAL_DEPENSES_COLLECTION_DECLARE',
        'LIGNE_29_CALC': 'L29_TOTAL_DEPENSES_COLLECTION_CALCULE',
        'ECART_LIGNE_29': 'L29_ECART',
        'MT_AID_SUBV_COLL': 'L30_SUBVENTIONS_COLLECTION',
        'MT_DEP_CONSEILS_CIR_COLL': 'L31_DEPENSES_CONSEIL_COLLECTION',
        'REMBST_SUBV_COLL': 'L32_REMBOURSEMENTS_SUBVENTIONS_COLL',
        'MT_NET_DEP_COLL': 'L33A_MONTANT_NET_COLLECTION_DECLARE',
        'LIGNE_33A_CALC': 'L33A_MONTANT_NET_COLLECTION_CALCULE',
        'ECART_LIGNE_33A': 'L33A_ECART',
        'MT_NET_DEP_COLL_DOM': 'L33B_MONTANT_NET_COLLECTION_DOM_DECLARE',
        'LIGNE_33B_CALC': 'L33B_MONTANT_NET_COLLECTION_DOM_CALCULE',
        'ECART_LIGNE_33B': 'L33B_ECART',
        'LIGNE_34A_CALC': 'L34A_MONTANT_NET_TOTAL_RD_COLL',
        'LIGNE_34B_CALC': 'L34B_MONTANT_NET_TOTAL_RD_COLL_DOM',

        # III. Calcul CIR Recherche et Collection
        'LIGNE_36_CALC': 'L36_CREDIT_IMPOT_RECHERCHE_MOINS_100M',
        'LIGNE_37': 'L37_QUOTE_PART_RECHERCHE_SOC_PERSONNES',
        'LIGNE_38A_CALC': 'L38A_CREDIT_IMPOT_RECHERCHE_TOTAL',
        'LIGNE_38B_CALC': 'L38B_CREDIT_IMPOT_RECHERCHE_DOM',
        'LIGNE_40_CALC': 'L40_CREDIT_IMPOT_COLLECTION_MOINS_100M',
        'LIGNE_41': 'L41_QUOTE_PART_COLLECTION_SOC_PERSONNES',
        'LIGNE_42A_CALC': 'L42A_CREDIT_IMPOT_COLL_AVANT_PLAF',
        'LIGNE_42B_CALC': 'L42B_CREDIT_IMPOT_COLL_DOM_AVANT_PLAF',
        'LIGNE_43': 'L43_AIDES_MINIMIS',
        'LIGNE_44': 'L44_CUMUL_CREDIT_IMPOT_ET_AIDES',
        'LIGNE_45A_CALC': 'L45A_CREDIT_IMPOT_COLL_APRES_PLAF',
        'LIGNE_45B_CALC': 'L45B_CREDIT_IMPOT_COLL_DOM_APRES_PLAF',
        'LIGNE_46A_CALC': 'L46A_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION',
        'LIGNE_46B_CALC': 'L46B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM',
        'LIGNE_47A_CALC': 'L47A_DEPENSES_RECHERCHE_LIMITE_100M',
        'LIGNE_47B_CALC': 'L47B_DEPENSES_RECHERCHE_DOM_LIMITE',
        'LIGNE_48_CALC': 'L48_CIR_RECHERCHE_PREMIERE_TRANCHE',
        'LIGNE_49_CALC': 'L49_DEPENSES_RECHERCHE_SUP_100M',
        'LIGNE_50_CALC': 'L50_CIR_RECHERCHE_DEUXIEME_TRANCHE',
        'LIGNE_51_CALC': 'L51_CIR_RECHERCHE_PLUS_100M',
        'LIGNE_52': 'L52_QUOTE_PART_RECHERCHE_SOC_PERSONNES_PLUS_100M',
        'LIGNE_53A_CALC': 'L53A_CREDIT_IMPOT_RECHERCHE_TOTAL_PLUS_100M',
        'LIGNE_53B_CALC': 'L53B_CREDIT_IMPOT_RECHERCHE_DOM_PLUS_100M',
        'LIGNE_54A_CALC': 'L54A_MONTANT_NET_COLLECTION_PLUS_100M',
        'LIGNE_54B_CALC': 'L54B_MONTANT_NET_COLLECTION_DOM_PLUS_100M',
        'LIGNE_55_CALC': 'L55_PLAFOND_DISPO_COLLECTION',
        'LIGNE_56_CALC': 'L56_CIR_COLLECTION_PREMIERE_TRANCHE',
        'LIGNE_57_CALC': 'L57_CIR_COLLECTION_DEUXIEME_TRANCHE',
        'LIGNE_58_CALC': 'L58_CIR_COLLECTION_PLUS_100M',
        'LIGNE_59': 'L59_QUOTE_PART_COLLECTION_SOC_PERSONNES_PLUS_100M',
        'LIGNE_60': 'L60_CREDIT_IMPOT_COLL_AVANT_PLAF_PLUS_100M',
        'LIGNE_61': 'L61_AIDES_MINIMIS_PLUS_100M',
        'LIGNE_62': 'L62_CUMUL_CREDIT_IMPOT_ET_AIDES_PLUS_100M',
        'LIGNE_63A_CALC': 'L63A_CREDIT_IMPOT_COLL_APRES_PLAF_PLUS_100M',
        'LIGNE_63B_CALC': 'L63B_CREDIT_IMPOT_COLL_DOM_APRES_PLAF_PLUS_100M',
        'LIGNE_64A_CALC': 'L64A_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_PLUS_100M',
        'LIGNE_64B_CALC': 'L64B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM_PLUS_100M',

        # IV. Dépenses d'innovation
        'DOT_AMORT_IMMO_INO': 'L65_DOTATION_AMORT_IMMO_INNOVATION',
        'DEP_PERSONEL_INO': 'L66_DEPENSES_PERSONNEL_INNOVATION',
        'LIGNE_67_CALC': 'L67_AUTRES_DEP_FONCT_INNOVATION',
        'FRAIS_BREV_COV_INO': 'L68_FRAIS_BREVETS_INNOVATION',
        'FRAIS_DEF_BREV_INO': 'L69_FRAIS_DEFENSE_BREVETS_INNOVATION',
        'OP_INOV_EXT': 'L70_OPERATIONS_CONFIEES_INNOVATION',
        'MT_TOT_DEP_INO': 'L71_TOTAL_DEPENSES_INNOVATION_DECLARE',
        'LIGNE_71_CALC': 'L71_TOTAL_DEPENSES_INNOVATION_CALCULE',
        'ECART_LIGNE_71': 'L71_ECART',
        'MT_TOT_DEP_INO_PLAF': 'L72_DEPENSES_INNOVATION_PLAFONNEES_DECLARE',
        'LIGNE_72_CALC': 'L72_DEPENSES_INNOVATION_PLAFONNEES_CALCULE',
        'ECART_LIGNE_72': 'L72_ECART',
        'MT_AID_SUBV_INO': 'L73_SUBVENTIONS_INNOVATION',
        'MT_ENC_PRESTA_INO': 'L74_PRESTATIONS_INNOVATION',
        'MT_DEP_CONSEILS_CII': 'L75_DEPENSES_CONSEIL_INNOVATION',
        'REMBST_SUBV_INO': 'L76_REMBOURSEMENTS_SUBVENTIONS_INNO',
        'MT_NET_DEP_INO': 'L77A_MONTANT_NET_INNOVATION_DECLARE',
        'LIGNE_77A_CALC': 'L77A_MONTANT_NET_INNOVATION_CALCULE',
        'ECART_LIGNE_77A': 'L77A_ECART',
        'MT_NET_DEP_INO_DOM': 'L77B_MONTANT_NET_INNOVATION_DOM',
        'MT_NET_DEP_INO_MPE_CORSE': 'L77C_MONTANT_NET_INNOVATION_CORSE_MPE',
        'MT_NET_DEP_INO_ME_CORSE': 'L77D_MONTANT_NET_INNOVATION_CORSE_ME',
        'LIGNE_78_CALC': 'L78_CREDIT_IMPOT_INNOVATION',
        'LIGNE_79': 'L79_QUOTE_PART_INNOVATION_SOC_PERSONNES',
        'LIGNE_80A_CALC': 'L80A_TOTAL_CREDIT_IMPOT_INNOVATION',
        'LIGNE_80B_CALC': 'L80B_CREDIT_IMPOT_INNOVATION_DOM',
        'LIGNE_80C_CALC': 'L80C_CREDIT_IMPOT_INNOVATION_CORSE',
        'LIGNE_81A_CALC': 'L81A_TOTAL_CIR_RECH_COLL_INNO',
        'LIGNE_81B_CALC': 'L81B_TOTAL_CIR_RECH_COLL_INNO_DOM',

        # V. Recherche collaborative
        'DEP_CRC': 'L82_DEPENSES_RECHERCHE_COLLABORATIVE',
        'DEP_CRC_FR': 'L83A_DEPENSES_RC_FRANCE',
        'DEP_CRC_ETR': 'L83B_DEPENSES_RC_ETRANGER',
        'LIGNE_83_CALC': 'L83_TOTAL_DEPENSES_RC',
        'DEP_CRC_PLAF': 'L84_DEPENSES_RC_PLAFONNEES_DECLARE',
        'LIGNE_84_CALC': 'L84_DEPENSES_RC_PLAFONNEES_CALCULE',
        'ECART_LIGNE_84': 'L84_ECART',
        'AIDE_PUB_CRC': 'L85_AIDES_PUBLIQUES_RC',
        'AIDE_PUB_REMB_CRC': 'L86_REMBOURSEMENTS_AIDES_RC',
        'MT_NET_DEP_CRC': 'L87_MONTANT_NET_RC_DECLARE',
        'MT_NET_DEP_CRC_CALC': 'L87_MONTANT_NET_RC_CALCULE',
        'ECART_MT_NET_DEP_CRC': 'L87_ECART',
        'MT_NET_DEP_CRC_PME': 'L88_MONTANT_NET_RC_PME',
        'LIGNE_89_CALC': 'L89_CREDIT_IMPOT_RC',
        'LIGNE_90': 'L90_QUOTE_PART_RC_SOC_PERSONNES',
        'LIGNE_91_CALC': 'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE',

        # Crédits d'impôt par composante
        'MT_CIR_RECH_YC_QP_MOINS_DE100': 'CIR_RECHERCHE_DECLARE',
        'CIR_RECHERCHE_RECALCULE': 'CIR_RECHERCHE_CALCULE',
        'ECART_CIR_RECHERCHE': 'CIR_RECHERCHE_ECART',

        'MT_CI_COLL_APRS_MINIMI': 'CIR_COLLECTION_DECLARE',
        'CIR_COLLECTION_RECALCULE': 'CIR_COLLECTION_CALCULE',
        'ECART_CIR_COLLECTION': 'CIR_COLLECTION_ECART',

        'MT_CII_YC_QP': 'CIR_INNOVATION_DECLARE',
        'CIR_INNOVATION_RECALCULE': 'CIR_INNOVATION_CALCULE',
        'ECART_CIR_INNOVATION': 'CIR_INNOVATION_ECART',

        'MT_CRC': 'CIR_COLLABORATIF_DECLARE',
        'CIR_COLLAB_RECALCULE': 'CIR_COLLABORATIF_CALCULE',
        'ECART_CIR_COLLAB': 'CIR_COLLABORATIF_ECART',

        # CIR Total
        'MT_TOT_CIR_CI_COLL_CII': 'CIR_TOTAL_DECLARE',
        'CIR_TOTAL_RECALCULE': 'CIR_TOTAL_CALCULE',
        'ECART_CIR': 'CIR_TOTAL_ECART',

        # Indicateurs
        'CORRESPONDANCE': 'CORRESPONDANCE_CIR',
        'ECART_RELATIF': 'ECART_RELATIF_POURCENT'
    }

    # Création du DataFrame de comparaison
    df_comparaison = pd.DataFrame()

    # Copier chaque colonne si elle existe
    for col_orig, col_dest in mapping_colonnes.items():
        if col_orig in df_num.columns:
            df_comparaison[col_dest] = df_num[col_orig]
        else:
            # Ne pas ajouter les colonnes qui n'existent pas
            pass

    # Concaténation de la désignation et du complément si disponible
    if 'COMPLEMENT_DESIGNATION' in df_comparaison.columns:
        df_comparaison['DESIGNATION'] = df_comparaison.apply(
            lambda x: f"{x['DESIGNATION']} {x['COMPLEMENT_DESIGNATION']}" if pd.notna(x['COMPLEMENT_DESIGNATION']) else x['DESIGNATION'],
            axis=1
        )
        df_comparaison.drop('COMPLEMENT_DESIGNATION', axis=1, inplace=True)

    # Formatage des montants
    montant_cols = [col for col in df_comparaison.columns if col not in ['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'DESIGNATION', 'CORRESPONDANCE_CIR']]
    for col in montant_cols:
        df_comparaison[col] = df_comparaison[col].round(2)

    # Sauvegarde du fichier de sortie
    output_filename = f"Calcul_Creance_CIR.csv"
    output_path = os.path.join("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB", output_filename)

    df_comparaison.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')

    print(f"\nFichier de comparaison détaillée créé: {output_path}")

    # RÉSULTATS

    # Somme des CIR déclarés
    cir_declare_total = df_tmp['MT_TOT_CIR_CI_COLL_CII'].sum()

    # Somme des CIR recalculés
    cir_recalcule_total = df_calc_final['CIR_TOTAL_RECALCULE'].sum()

    # Nombre d'entreprises
    nb_entreprises_avec_cir = len(df_tmp[df_tmp['MT_TOT_CIR_CI_COLL_CII'] > 0])
    nb_entreprises_plus_100m = df_calc_final['DEPENSES_PLUS_100M'].sum()

    # Classement des écarts
    ecarts_positifs = df_num[df_num['ECART_CIR'] > 1]  # Plus de 1 en plus
    ecarts_negatifs = df_num[df_num['ECART_CIR'] < -1]  # Plus de 1 en moins
    ecarts_conformes = df_num[abs(df_num['ECART_CIR']) <= 1]  # Différence de 1 ou moins

    # AFFICHAGE DES RÉSULTATS
    print("\n===== COMPARAISON CIR DÉCLARÉ VS RECALCULÉ =====")

    print(f"\nNombre total d'entreprises analysées: {len(df_num):,}")
    print(f"Nombre d'entreprises avec CIR déclaré: {nb_entreprises_avec_cir:,}")
    print(f"Nombre d'entreprises avec dépenses > 100M€: {nb_entreprises_plus_100m:,}")

    print(f"\nCIR TOTAL:")
    print(f"CIR déclaré total: {cir_declare_total:,.2f} €")
    print(f"CIR recalculé total: {cir_recalcule_total:,.2f} €")

    difference = cir_recalcule_total - cir_declare_total
    print(f"Différence: {difference:,.2f} €")
    if cir_declare_total > 0:
        print(f"Écart relatif: {difference/cir_declare_total*100:.2f}%")

    print(f"\nANALYSE DES ÉCARTS:")
    print(f"Entreprises avec CIR conforme (écart ≤ 1€): {len(ecarts_conformes):,} ({len(ecarts_conformes)/len(df_num)*100:.2f}%)")

    print(f"Entreprises avec CIR recalculé > CIR déclaré (écart > 1€): {len(ecarts_positifs):,}")
    print(f"Montant total des écarts positifs: {ecarts_positifs['ECART_CIR'].sum():,.2f} €")

    print(f"Entreprises avec CIR recalculé < CIR déclaré (écart < -1€): {len(ecarts_negatifs):,}")
    print(f"Montant total des écarts négatifs: {ecarts_negatifs['ECART_CIR'].sum():,.2f} €")

    # Détail par composante du CIR
    print(f"\nDÉTAIL PAR COMPOSANTE DU CIR RECALCULÉ:")
    print(f"CIR Recherche: {df_num['CIR_RECHERCHE_RECALCULE'].sum():,.2f} €")
    print(f"CIR Collection: {df_num['CIR_COLLECTION_RECALCULE'].sum():,.2f} €")

    if 'CIR_INNOVATION_RECALCULE' in df_num.columns:
        print(f"CIR Innovation: {df_num['CIR_INNOVATION_RECALCULE'].sum():,.2f} €")
    else:
        print(f"CIR Innovation: 0.00 €")

    if 'CIR_COLLAB_RECALCULE' in df_num.columns:
        print(f"CIR Recherche Collaborative: {df_num['CIR_COLLAB_RECALCULE'].sum():,.2f} €")
    else:
        print(f"CIR Recherche Collaborative: 0.00 €")

    return df_comparaison

# Exécuter le calcul
try:
    df_resultat = comparer_cir_declare_recalcule()
    print("\nTraitement terminé avec succès!")
except Exception as e:
    print(f"\nErreur: {type(e).__name__}: {str(e)}")

Nombre total d'entreprises: 27,957


C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\3507620946.py:1002: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_comparaison[col_dest] = df_num[col_orig]
C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\3507620946.py:1002: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_comparaison[col_dest] = df_num[col_orig]
C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\3507620946.py:1002: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance


Fichier de comparaison détaillée créé: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB\Calcul_Creance_CIR.csv

===== COMPARAISON CIR DÉCLARÉ VS RECALCULÉ =====

Nombre total d'entreprises analysées: 27,957
Nombre d'entreprises avec CIR déclaré: 27,233
Nombre d'entreprises avec dépenses > 100M€: 21

CIR TOTAL:
CIR déclaré total: 8,874,600,095.00 €
CIR recalculé total: 7,586,181,391.68 €
Différence: -1,288,418,703.32 €
Écart relatif: -14.52%

ANALYSE DES ÉCARTS:
Entreprises avec CIR conforme (écart ≤ 1€): 26,408 (94.46%)
Entreprises avec CIR recalculé > CIR déclaré (écart > 1€): 541
Montant total des écarts positifs: 30,076,612.92 €
Entreprises avec CIR recalculé < CIR déclaré (écart < -1€): 1,008
Montant total des écarts négatifs: -1,318,495,541.68 €

DÉTAIL PAR COMPOSANTE DU CIR RECALCULÉ:
CIR Recherche: 7,148,118,828.26 €
CIR Collection: 30,130,760.00 €
CIR Innovation: 399,154,891.22 €
CIR Recherche Collaborative: 8,776,912.20 €

Traitement terminé avec succès!


# Difference Creance Déclarer vs Calculé

## Crc manquant


In [31]:
import pandas as pd
import numpy as np

# Fonction pour formater les SIREN correctement
def format_siren(siren):
    """Formate les SIREN pour avoir exactement 9 caractères en ajoutant des zéros au début si nécessaire"""
    if pd.isna(siren):
        return siren

    siren_str = str(siren).strip()

    # Si le SIREN a moins de 9 caractères, ajouter des zéros au début
    if len(siren_str) < 9:
        siren_str = siren_str.zfill(9)  # zfill ajoute des zéros au début jusqu'à atteindre la longueur spécifiée

    return siren_str

# Charger le fichier CSV avec converters pour traiter SIREN comme du texte
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig',
                converters={'SIREN_DECLARANT': str, 'SIREN_DEPOSANT': str})

print(f"Nombre total d'entreprises: {len(df)}")
print(f"Nombre d'entreprises avec écarts (CORRESPONDANCE_CIR = 'Non'): {len(df[df['CORRESPONDANCE_CIR'] == 'Non'])}")

# 1. Corriger le format des SIREN
if 'SIREN_DECLARANT' in df.columns:
    # Afficher quelques exemples de SIREN avant correction
    print("\nExemples de SIREN_DECLARANT avant correction:")
    for siren in df['SIREN_DECLARANT'].head(5):
        print(f"  {siren} (type: {type(siren)}, longueur: {len(str(siren))})")

    # Identifier les SIREN qui n'ont pas 9 caractères
    siren_lengths = df['SIREN_DECLARANT'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
    unusual_siren_count = (siren_lengths != 9).sum()
    print(f"\nNombre de SIREN n'ayant pas exactement 9 caractères: {unusual_siren_count}")

    # Appliquer la correction aux SIREN
    df['SIREN_DECLARANT'] = df['SIREN_DECLARANT'].apply(format_siren)

    # Si la colonne SIREN_DEPOSANT existe également, la corriger
    if 'SIREN_DEPOSANT' in df.columns:
        df['SIREN_DEPOSANT'] = df['SIREN_DEPOSANT'].apply(format_siren)

    # Vérifier les SIREN après correction
    print("\nExemples de SIREN_DECLARANT après correction:")
    for siren in df['SIREN_DECLARANT'].head(5):
        print(f"  {siren} (type: {type(siren)}, longueur: {len(str(siren))})")

    # Vérifier qu'il n'y a plus de SIREN avec une longueur incorrecte
    siren_lengths_after = df['SIREN_DECLARANT'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
    unusual_siren_count_after = (siren_lengths_after != 9).sum()
    print(f"Nombre de SIREN n'ayant pas exactement 9 caractères après correction: {unusual_siren_count_after}")

# 2. Identifier les cas où l'écart est dû au CRC manquant
tolerance = 1.0  # 1€ de tolérance

# Créer un masque pour identifier les cas où l'écart correspond au CRC calculé
mask_crc_missing = (
    # Cas où l'écart est positif (montant calculé > montant déclaré)
    (df['CIR_TOTAL_ECART'] > 0) &
    # L'écart est approximativement égal au CRC calculé
    (abs(df['CIR_TOTAL_ECART'] - df['CIR_COLLABORATIF_CALCULE']) < tolerance) &
    # Le CRC déclaré est nul ou très petit comparé au CRC calculé
    ((df['CIR_COLLABORATIF_DECLARE'].isna()) |
     (df['CIR_COLLABORATIF_DECLARE'] == 0) |
     (df['CIR_COLLABORATIF_CALCULE'] > df['CIR_COLLABORATIF_DECLARE'] * 1.5))
)

# Compter les cas identifiés
count_crc_missing = mask_crc_missing.sum()
print(f"\nNombre d'entreprises avec CRC apparemment manquant dans le montant déclaré: {count_crc_missing}")

# 3. Sauvegarder les valeurs originales (pour information uniquement)
df['CIR_TOTAL_DECLARE_ORIGINAL'] = df['CIR_TOTAL_DECLARE'].copy()
df['CIR_TOTAL_ECART_ORIGINAL'] = df['CIR_TOTAL_ECART'].copy()
df['CORRESPONDANCE_CIR_ORIGINAL'] = df['CORRESPONDANCE_CIR'].copy()

# 4. Corriger directement le montant déclaré en ajoutant le CRC calculé
df.loc[mask_crc_missing, 'CIR_TOTAL_DECLARE'] = df.loc[mask_crc_missing, 'CIR_TOTAL_DECLARE'] + df.loc[mask_crc_missing, 'CIR_COLLABORATIF_CALCULE']

# 5. Recalculer l'écart
df['CIR_TOTAL_ECART'] = df['CIR_TOTAL_CALCULE'] - df['CIR_TOTAL_DECLARE']

# 6. Mettre à jour la correspondance
df['CORRESPONDANCE_CIR'] = np.where(abs(df['CIR_TOTAL_ECART']) <= 1, "Oui", "Non")

# 7. Analyser les résultats
count_original_non = (df['CORRESPONDANCE_CIR_ORIGINAL'] == 'Non').sum()
count_corrected_non = (df['CORRESPONDANCE_CIR'] == 'Non').sum()
count_fixed = count_original_non - count_corrected_non

print(f"\nRésultats après correction:")
print(f"Nombre d'entreprises avec écarts avant correction: {count_original_non}")
print(f"Nombre d'entreprises avec écarts après correction: {count_corrected_non}")
print(f"Nombre d'écarts résolus par l'ajout du CRC: {count_fixed} ({count_fixed/count_original_non*100:.2f}%)")

# 8. Exemples d'entreprises corrigées (pour information uniquement)
print("\nExemples d'entreprises où l'écart a été résolu par l'ajout du CRC:")
df_resolved = df[(df['CORRESPONDANCE_CIR_ORIGINAL'] == 'Non') & (df['CORRESPONDANCE_CIR'] == 'Oui')].copy()
if len(df_resolved) > 0:
    sample_size = min(3, len(df_resolved))
    for i, (_, row) in enumerate(df_resolved.head(sample_size).iterrows(), 1):
        print(f"\nExemple {i}: {row.get('DESIGNATION', 'N/A')} (SIREN: {row.get('SIREN_DECLARANT', 'N/A')})")
        print(f"  CIR déclaré original: {row['CIR_TOTAL_DECLARE_ORIGINAL']:.2f}")
        print(f"  CIR déclaré corrigé: {row['CIR_TOTAL_DECLARE']:.2f}")
        print(f"  CIR calculé: {row['CIR_TOTAL_CALCULE']:.2f}")
        print(f"  Écart original: {row['CIR_TOTAL_ECART_ORIGINAL']:.2f}")
        print(f"  Écart corrigé: {row['CIR_TOTAL_ECART']:.2f}")
        print(f"  CRC calculé: {row.get('CIR_COLLABORATIF_CALCULE', 'N/A')}")

# 9. Supprimer les colonnes originales pour ne garder que les valeurs corrigées
df = df.drop(['CIR_TOTAL_DECLARE_ORIGINAL', 'CIR_TOTAL_ECART_ORIGINAL', 'CORRESPONDANCE_CIR_ORIGINAL'], axis=1)

# 10. Écraser le fichier original avec les données corrigées
# S'assurer que les SIREN sont toujours formatés comme du texte lors de l'export
df.to_csv(file_path, sep=';', encoding='utf-8-sig', index=False)

print(f"\nFichier original mis à jour: {file_path}")
print(f"- Format des SIREN corrigé")
print(f"- CRC manquant ajouté au montant déclaré")

# 11. Résumé des écarts restants
if count_corrected_non > 0:
    print(f"\nIl reste {count_corrected_non} entreprises avec des écarts à analyser.")

    # Identifier les principales sources d'écarts restants
    df_remaining_ecarts = df[df['CORRESPONDANCE_CIR'] == 'Non']

    # Obtenir toutes les colonnes d'écart
    ecart_cols = [col for col in df.columns if '_ECART' in col and col != 'CIR_TOTAL_ECART' and col != 'ECART_RELATIF_POURCENT']

    # Compter les écarts significatifs par colonne
    ecart_counts = {}
    for col in ecart_cols:
        if col in df_remaining_ecarts.columns:
            non_zero = df_remaining_ecarts[abs(df_remaining_ecarts[col]) > 1]
            if len(non_zero) > 0:
                ecart_counts[col] = len(non_zero)

    # Afficher les 5 principales sources d'écarts
    sorted_ecarts = sorted(ecart_counts.items(), key=lambda x: x[1], reverse=True)
    if sorted_ecarts:
        print("\nPrincipales sources d'écarts restants:")
        for i, (col, count) in enumerate(sorted_ecarts[:5], 1):
            pct = count / count_corrected_non * 100
            print(f"{i}. {col}: {count} entreprises ({pct:.2f}%)")

    # Rechercher d'autres cas potentiels similaires au problème du CRC
    print("\nRecherche d'autres patterns d'écarts similaires:")

    # 1. Cas où l'écart correspond au CIR Innovation
    if 'CIR_INNOVATION_CALCULE' in df_remaining_ecarts.columns:
        innovation_missing = (
            (df_remaining_ecarts['CIR_TOTAL_ECART'] > 0) &
            (abs(df_remaining_ecarts['CIR_TOTAL_ECART'] - df_remaining_ecarts['CIR_INNOVATION_CALCULE']) < tolerance)
        )
        count_innovation = innovation_missing.sum()
        if count_innovation > 0:
            print(f"  • Possible CIR Innovation manquant: {count_innovation} entreprises")

    # 2. Cas où l'écart correspond au CIR Collection
    if 'CIR_COLLECTION_CALCULE' in df_remaining_ecarts.columns:
        collection_missing = (
            (df_remaining_ecarts['CIR_TOTAL_ECART'] > 0) &
            (abs(df_remaining_ecarts['CIR_TOTAL_ECART'] - df_remaining_ecarts['CIR_COLLECTION_CALCULE']) < tolerance)
        )
        count_collection = collection_missing.sum()
        if count_collection > 0:
            print(f"  • Possible CIR Collection manquant: {count_collection} entreprises")

Nombre total d'entreprises: 27957
Nombre d'entreprises avec écarts (CORRESPONDANCE_CIR = 'Non'): 1549

Exemples de SIREN_DECLARANT avant correction:
  005520325 (type: <class 'str'>, longueur: 9)
  005580436 (type: <class 'str'>, longueur: 9)
  005680145 (type: <class 'str'>, longueur: 9)
  006380042 (type: <class 'str'>, longueur: 9)
  006580195 (type: <class 'str'>, longueur: 9)

Nombre de SIREN n'ayant pas exactement 9 caractères: 0

Exemples de SIREN_DECLARANT après correction:
  005520325 (type: <class 'str'>, longueur: 9)
  005580436 (type: <class 'str'>, longueur: 9)
  005680145 (type: <class 'str'>, longueur: 9)
  006380042 (type: <class 'str'>, longueur: 9)
  006580195 (type: <class 'str'>, longueur: 9)
Nombre de SIREN n'ayant pas exactement 9 caractères après correction: 0

Nombre d'entreprises avec CRC apparemment manquant dans le montant déclaré: 12412

Résultats après correction:
Nombre d'entreprises avec écarts avant correction: 1549
Nombre d'entreprises avec écarts après

## Erreur depense personnel

In [32]:
import pandas as pd
import numpy as np

# Charger le fichier original
df = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv", sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Identifier les lignes concernées (ajout du critère correspondance)
mask = (
    (df['L3_DEPENSES_PERSONNEL_CHERCHEURS'] == 0.0) &
    (df['L2_DOTATION_AMORT_SINISTR'] != 0.0) &
    (df['CORRESPONDANCE_CIR'] == "Non")
)
indices_to_update = df[mask].index
print(f"Nombre de lignes à corriger: {len(indices_to_update)}")


# Sauvegarder les valeurs originales pour analyse
cir_calcule_avant = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].copy()

# Pour chaque ligne à corriger
for idx in indices_to_update:
    # 1. Transférer la valeur de dotation aux amortissements sinistrés vers dépenses de personnel
    val_amort_sinistr = df.at[idx, 'L2_DOTATION_AMORT_SINISTR']
    df.at[idx, 'L3_DEPENSES_PERSONNEL_CHERCHEURS'] = val_amort_sinistr
    df.at[idx, 'L2_DOTATION_AMORT_SINISTR'] = 0.0

    # 2. Recalculer la ligne 6 (autres dépenses de fonctionnement)
    dot_amort_immo = df.at[idx, 'L1_DOTATION_AMORT_IMMO']
    dep_cherch_tech = df.at[idx, 'L3_DEPENSES_PERSONNEL_CHERCHEURS']  # Nouvelle valeur
    rem_sal_inv = df.at[idx, 'L4_REMUNERATION_INVENTEURS']
    dep_jd = df.at[idx, 'L5_DEPENSES_JEUNES_DOCTEURS']

    autres_dep_fonct = (dot_amort_immo * 0.75) + ((dep_cherch_tech + rem_sal_inv) * 0.43) + dep_jd
    df.at[idx, 'L6_AUTRES_DEP_FONCT_CALCULE'] = autres_dep_fonct

    # 3. Recalculer la ligne 7 (total dépenses de fonctionnement)
    total_dep_fonct = dot_amort_immo + 0.0 + dep_cherch_tech + rem_sal_inv + dep_jd + autres_dep_fonct
    df.at[idx, 'L7_TOTAL_DEP_FONCT_CALCULE'] = total_dep_fonct

    # 4. Recalculer la ligne 14 (total dépenses internes)
    frais_brev_cov = df.at[idx, 'L8_FRAIS_BREVETS_COV']
    dep_maint_brev_cov = df.at[idx, 'L9_DEPENSES_DEFENSE_BREVETS']
    dot_amort_brev = df.at[idx, 'L10_DOTATION_AMORT_BREVETS']
    dep_normali = df.at[idx, 'L11_DEPENSES_NORMALISATION_DECLARE']
    prim_cotiz = min(df.at[idx, 'L12_PRIMES_COTISATIONS_BRUT'], 60000)
    dep_veil_techno = min(df.at[idx, 'L13_VEILLE_TECHNO_BRUT'], 60000)

    total_dep_internes = total_dep_fonct + frais_brev_cov + dep_maint_brev_cov + dot_amort_brev + dep_normali + prim_cotiz + dep_veil_techno
    df.at[idx, 'L14_TOTAL_DEPENSES_INTERNES_CALCULE'] = total_dep_internes

    # 5. Recalculer la ligne 22 (total dépenses de recherche)
    total_dep_ext = df.at[idx, 'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE']
    total_dep_recherche = total_dep_internes + total_dep_ext
    df.at[idx, 'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE'] = total_dep_recherche

    # 6. Recalculer la ligne 26A (montant net des dépenses)
    mt_aid_subv = df.at[idx, 'L23A_SUBVENTIONS']
    mt_enc_presta = df.at[idx, 'L23B_SOMMES_ENCAISSEES_TIERS']
    mt_dep_conseils_cir = df.at[idx, 'L24_DEPENSES_CONSEIL_CIR']
    rembst_subv = df.at[idx, 'L25_REMBOURSEMENTS_SUBVENTIONS']

    montant_net = total_dep_recherche - mt_aid_subv - mt_enc_presta - mt_dep_conseils_cir + rembst_subv
    df.at[idx, 'L26A_MONTANT_NET_DEPENSES_CALCULE'] = montant_net

    # 7. Recalculer le CIR recherche (30% du montant net)
    cir_recherche = montant_net * 0.30
    df.at[idx, 'CIR_RECHERCHE_CALCULE'] = cir_recherche

    # 8. Recalculer le CIR total
    cir_collection = df.at[idx, 'CIR_COLLECTION_CALCULE']
    cir_innovation = df.at[idx, 'CIR_INNOVATION_CALCULE']
    cir_collaboratif = df.at[idx, 'CIR_COLLABORATIF_CALCULE']

    cir_total = cir_recherche + cir_collection + cir_innovation + cir_collaboratif
    df.at[idx, 'CIR_TOTAL_CALCULE'] = cir_total

    # 9. Recalculer l'écart et mettre à jour la correspondance
    ecart_cir = cir_total - df.at[idx, 'CIR_TOTAL_DECLARE']
    df.at[idx, 'CIR_TOTAL_ECART'] = ecart_cir
    df.at[idx, 'CORRESPONDANCE_CIR'] = "Oui" if abs(ecart_cir) <= 1 else "Non"

# Afficher les résultats
somme_declare = df.loc[indices_to_update, 'CIR_TOTAL_DECLARE'].sum()
somme_calcule_avant = cir_calcule_avant.sum()
somme_calcule_apres = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].sum()

print(f"\nRésultats de la correction:")
print(f"Somme CIR_TOTAL_DECLARE: {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE avant correction: {somme_calcule_avant:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE après correction: {somme_calcule_apres:,.2f} €")
print(f"Écart avant correction: {somme_calcule_avant - somme_declare:,.2f} €")
print(f"Écart après correction: {somme_calcule_apres - somme_declare:,.2f} €")

# Sauvegarder le fichier corrigé
df.to_csv("lignes_restantes_a_analyser_corrigees.csv", sep=';', encoding='utf-8-sig', index=False)
print(f"\nFichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees.csv")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\3028750523.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv", sep=';', encoding='utf-8-sig')


Fichier chargé: 27957 lignes
Nombre de lignes à corriger: 16

Résultats de la correction:
Somme CIR_TOTAL_DECLARE: 7,170,745.00 €
Somme CIR_TOTAL_CALCULE avant correction: 6,298,270.92 €
Somme CIR_TOTAL_CALCULE après correction: 7,706,819.31 €
Écart avant correction: -872,474.08 €
Écart après correction: 536,074.31 €

Fichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees.csv

Fichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees.csv


### mise à jour ligne restante

In [33]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv", sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Appliquer les filtres :
# - L5_DEPENSES_JEUNES_DOCTEURS = 0.0
# - L2_DOTATION_AMORT_SINISTR différent de 0.0
df_filtered = df[
    (df['L3_DEPENSES_PERSONNEL_CHERCHEURS'] == 0.0) &
    (df['L2_DOTATION_AMORT_SINISTR'] != 0.0) &
    (df['CORRESPONDANCE_CIR'] == "Non") 
]

print(f"Nombre de lignes après filtrage: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_jeunes_docteurs_0_amort_sinistr_non0.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

# Afficher quelques statistiques sur les lignes trouvées
if len(df_filtered) > 0:
    print("\nAperçu des données trouvées:")
    print(f"Nombre d'entreprises: {len(df_filtered)}")

Fichier chargé: 27957 lignes
Nombre de lignes après filtrage: 16
Somme CIR_TOTAL_DECLARE : 7,170,745.00 €
Somme CIR_TOTAL_CALCULE : 6,298,270.92 €
Résultats exportés dans: lignes_jeunes_docteurs_0_amort_sinistr_non0.csv

Aperçu des données trouvées:
Nombre d'entreprises: 16


C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\585773094.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv", sep=';', encoding='utf-8-sig')


In [35]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')  # Nouveau fichier

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])  # Nouveau set

# Combiner tous les SIREN à exclure
siren_a_exclure =  siren_jeunes_docteurs  # Ajout du nouveau set

# Filtrer le DataFrame principal
# - Exclure les SIREN déjà analysés
# - Garder uniquement les lignes où CORRESPONDANCE_CIR est "Non"
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) & 
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\2105814575.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 1514
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur DEP Fonctionnement

In [36]:
import pandas as pd
import numpy as np

# Charger le fichier original
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Identifier les lignes à corriger
mask = (df['L7_TOTAL_DEP_FONCT_DECLARE'] == df['L8_FRAIS_BREVETS_COV']) & (df['L7_TOTAL_DEP_FONCT_DECLARE'] != 0.0)
indices_to_update = df[mask].index
print(f"Nombre de lignes à corriger: {len(indices_to_update)}")

# Sauvegarder les valeurs originales pour analyse
cir_calcule_avant = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].copy()
l7_original = df.loc[indices_to_update, 'L7_TOTAL_DEP_FONCT_DECLARE'].copy()

# Pour chaque ligne à corriger
for idx in indices_to_update:
    # 1. Calculer correctement le montant de la ligne 7 (total dépenses de fonctionnement)
    dot_amort_immo = df.at[idx, 'L1_DOTATION_AMORT_IMMO']
    dot_amort_sinistr = df.at[idx, 'L2_DOTATION_AMORT_SINISTR']
    dep_cherch_tech = df.at[idx, 'L3_DEPENSES_PERSONNEL_CHERCHEURS']
    rem_sal_inv = df.at[idx, 'L4_REMUNERATION_INVENTEURS']
    dep_jd = df.at[idx, 'L5_DEPENSES_JEUNES_DOCTEURS']

    # Calculer L6 si nécessaire
    autres_dep_fonct = (dot_amort_immo * 0.75) + ((dep_cherch_tech + rem_sal_inv) * 0.43) + dep_jd

    # Mettre à jour L6 calculé
    df.at[idx, 'L6_AUTRES_DEP_FONCT_CALCULE'] = autres_dep_fonct

    # Calculer le vrai montant de L7
    total_dep_fonct = dot_amort_immo + dot_amort_sinistr + dep_cherch_tech + rem_sal_inv + dep_jd + autres_dep_fonct

    # 2. Mettre à jour L7
    df.at[idx, 'L7_TOTAL_DEP_FONCT_CALCULE'] = total_dep_fonct # laisser L7 tel quel (inchangé)

    # Garder L8_FRAIS_BREVETS_COV tel quel (inchangé)
    frais_brev_cov = df.at[idx, 'L8_FRAIS_BREVETS_COV'] # mettre à 0

    # 3. Recalculer la ligne 14 (total dépenses internes)
    dep_maint_brev_cov = df.at[idx, 'L9_DEPENSES_DEFENSE_BREVETS']
    dot_amort_brev = df.at[idx, 'L10_DOTATION_AMORT_BREVETS']
    dep_normali = df.at[idx, 'L11_DEPENSES_NORMALISATION_DECLARE']
    prim_cotiz = df.at[idx, 'L12_PRIMES_COTISATIONS_PLAFONNEES']
    dep_veil_techno = df.at[idx, 'L13_VEILLE_TECHNO_PLAFONNEE']

    total_dep_internes = total_dep_fonct + frais_brev_cov + dep_maint_brev_cov + dot_amort_brev + dep_normali + prim_cotiz + dep_veil_techno
    df.at[idx, 'L14_TOTAL_DEPENSES_INTERNES_CALCULE'] = total_dep_internes

    # 4. Recalculer la ligne 22 (total dépenses de recherche)
    total_dep_ext = df.at[idx, 'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE']
    total_dep_recherche = total_dep_internes + total_dep_ext
    df.at[idx, 'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE'] = total_dep_recherche

    # 5. Recalculer la ligne 26A (montant net des dépenses)
    mt_aid_subv = df.at[idx, 'L23A_SUBVENTIONS']
    mt_enc_presta = df.at[idx, 'L23B_SOMMES_ENCAISSEES_TIERS']
    mt_dep_conseils_cir = df.at[idx, 'L24_DEPENSES_CONSEIL_CIR']
    rembst_subv = df.at[idx, 'L25_REMBOURSEMENTS_SUBVENTIONS']

    montant_net = total_dep_recherche - mt_aid_subv - mt_enc_presta - mt_dep_conseils_cir + rembst_subv
    df.at[idx, 'L26A_MONTANT_NET_DEPENSES_CALCULE'] = montant_net

    # 6. Récupérer le montant DOM pour le calcul du CIR
    montant_net_dom = df.at[idx, 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE']

    # 7. Recalculer le CIR recherche (30% ou 50% selon localisation)
    cir_recherche = ((montant_net - montant_net_dom) * 0.30) + (montant_net_dom * 0.50)
    df.at[idx, 'CIR_RECHERCHE_CALCULE'] = cir_recherche

    # 8. Recalculer le CIR total
    cir_collection = df.at[idx, 'CIR_COLLECTION_CALCULE']
    cir_innovation = df.at[idx, 'CIR_INNOVATION_CALCULE']
    cir_collaboratif = df.at[idx, 'CIR_COLLABORATIF_CALCULE']

    cir_total = cir_recherche + cir_collection + cir_innovation + cir_collaboratif
    df.at[idx, 'CIR_TOTAL_CALCULE'] = cir_total

    # 9. Recalculer l'écart et mettre à jour la correspondance
    ecart_cir = cir_total - df.at[idx, 'CIR_TOTAL_DECLARE']
    df.at[idx, 'CIR_TOTAL_ECART'] = ecart_cir
    df.at[idx, 'CORRESPONDANCE_CIR'] = "Oui" if abs(ecart_cir) <= 1 else "Non"

# Afficher le résumé des modifications
l7_corrige = df.loc[indices_to_update, 'L7_TOTAL_DEP_FONCT_CALCULE']
somme_declare = df.loc[indices_to_update, 'CIR_TOTAL_DECLARE'].sum()
somme_calcule_avant = cir_calcule_avant.sum()
somme_calcule_apres = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].sum()

print(f"\nRésumé des corrections :")
print(f"Somme L7_TOTAL_DEP_FONCT_DECLARE avant correction: {l7_original.sum():,.2f} €")
print(f"Somme L7_TOTAL_DEP_FONCT_CALCULE après correction: {l7_corrige.sum():,.2f} €")
print(f"Différence sur L7: {l7_corrige.sum() - l7_original.sum():,.2f} €")

print(f"\nImpact sur le CIR :")
print(f"Somme CIR_TOTAL_DECLARE: {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE avant correction: {somme_calcule_avant:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE après correction: {somme_calcule_apres:,.2f} €")
print(f"Écart avant correction: {somme_calcule_avant - somme_declare:,.2f} €")
print(f"Écart après correction: {somme_calcule_apres - somme_declare:,.2f} €")
print(f"Amélioration de l'écart: {abs(somme_calcule_avant - somme_declare) - abs(somme_calcule_apres - somme_declare):,.2f} €")

# Compter les lignes où la correspondance s'est améliorée
count_improved = 0
for idx in indices_to_update:
    if df.at[idx, 'CORRESPONDANCE_CIR'] == "Oui" and abs(cir_calcule_avant[idx] - df.at[idx, 'CIR_TOTAL_DECLARE']) > 1:
        count_improved += 1

print(f"\nNombre de lignes où la correspondance est passée de 'Non' à 'Oui': {count_improved}")

# Sauvegarder le fichier corrigé
df.to_csv("lignes_restantes_a_analyser_corrigees_L7L8.csv", sep=';', encoding='utf-8-sig', index=False)
print(f"\nFichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees_L7L8.csv")

# Afficher des exemples de modifications
print("\nExemples de modifications significatives:")
# Calculer l'impact en pourcentage sur le CIR
df_impact = pd.DataFrame({
    'SIREN': df.loc[indices_to_update, 'SIREN_DECLARANT'],
    'Désignation': df.loc[indices_to_update, 'DESIGNATION'],
    'L7 avant': l7_original,
    'L7 après': l7_corrige,
    'CIR avant': cir_calcule_avant,
    'CIR après': df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'],
    'Impact': abs(df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'] - cir_calcule_avant)
})

# Afficher les 5 cas avec le plus grand impact en valeur absolue
for _, row in df_impact.sort_values('Impact', ascending=False).head(5).iterrows():
    print(f"SIREN: {row['SIREN']}")
    print(f"  Désignation: {row['Désignation']}")
    print(f"  L7 avant: {row['L7 avant']:,.2f} €")
    print(f"  L7 après: {row['L7 après']:,.2f} €")
    print(f"  CIR avant: {row['CIR avant']:,.2f} €")
    print(f"  CIR après: {row['CIR après']:,.2f} €")
    print(f"  Impact: {row['Impact']:,.2f} € ({row['Impact']/row['CIR avant']*100 if row['CIR avant'] != 0 else 0:.2f}%)")
    print()

Fichier chargé: 1514 lignes
Nombre de lignes à corriger: 1

Résumé des corrections :
Somme L7_TOTAL_DEP_FONCT_DECLARE avant correction: 6,056,576.00 €
Somme L7_TOTAL_DEP_FONCT_CALCULE après correction: 6,056,575.55 €
Différence sur L7: -0.45 €

Impact sur le CIR :
Somme CIR_TOTAL_DECLARE: 1,846,082.00 €
Somme CIR_TOTAL_CALCULE avant correction: 3,663,054.16 €
Somme CIR_TOTAL_CALCULE après correction: 3,663,054.17 €
Écart avant correction: 1,816,972.16 €
Écart après correction: 1,816,972.17 €
Amélioration de l'écart: -0.00 €

Nombre de lignes où la correspondance est passée de 'Non' à 'Oui': 0

Fichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees_L7L8.csv

Exemples de modifications significatives:
SIREN: 521474445
  Désignation: TEHTRI-SECURITY SAS NS
  L7 avant: 6,056,576.00 €
  L7 après: 6,056,575.55 €
  CIR avant: 3,663,054.16 €
  CIR après: 3,663,054.17 €
  Impact: 0.00 € (0.00%)


Fichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees_L7L8.csv

Exemples de m

### ligne liée au erreur dep fonctionnement

In [37]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où L7_TOTAL_DEP_FONCT_DECLARE = L8_FRAIS_BREVETS_COV
df_filtered = df[(df['L7_TOTAL_DEP_FONCT_DECLARE'] == df['L8_FRAIS_BREVETS_COV']) & (df['L7_TOTAL_DEP_FONCT_DECLARE'] != 0.0)]

print(f"Nombre de lignes où L7_TOTAL_DEP_FONCT_DECLARE = L8_FRAIS_BREVETS_COV: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_L7_egal_L8.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 1514 lignes
Nombre de lignes où L7_TOTAL_DEP_FONCT_DECLARE = L8_FRAIS_BREVETS_COV: 1
Somme CIR_TOTAL_DECLARE : 1,846,082.00 €
Somme CIR_TOTAL_CALCULE : 3,663,054.16 €
Résultats exportés dans: lignes_L7_egal_L8.csv


In [38]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')  # Nouveau fichier
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT']) 
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure
siren_a_exclure =  siren_jeunes_docteurs | siren_l7_l8 

# Filtrer le DataFrame principal
# - Exclure les SIREN déjà analysés
# - Garder uniquement les lignes où CORRESPONDANCE_CIR est "Non"
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) & 
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\3477145.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 1513
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir Recherche

In [39]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("lignes_restantes_a_analyser.csv", sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Appliquer les filtres :
# - L22_TOTAL_DEPENSES_RECHERCHE_DECLARE différent de 0.0
# - CIR_RECHERCHE_DECLARE = 0.0
df_filtered = df[
    (df['L22_TOTAL_DEPENSES_RECHERCHE_DECLARE'] != 0.0) &
    (df['CIR_RECHERCHE_DECLARE'] == 0.0)
]

print(f"Nombre de lignes après filtrage: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_depenses_non0_cir_recherche_0.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

# Afficher quelques statistiques sur les lignes trouvées
if len(df_filtered) > 0:
    print("\nAperçu des données trouvées:")
    print(f"Nombre d'entreprises: {len(df_filtered)}")

Fichier chargé: 1513 lignes
Nombre de lignes après filtrage: 23
Somme CIR_TOTAL_DECLARE : 95,803,865.00 €
Somme CIR_TOTAL_CALCULE : 102,591,496.33 €
Résultats exportés dans: lignes_depenses_non0_cir_recherche_0.csv

Aperçu des données trouvées:
Nombre d'entreprises: 23


In [40]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du nouveau cas)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8  | siren_depenses_non0_cir0

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\661988201.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 1490
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur depense Externalisée

In [41]:
import pandas as pd

# Chemin du fichier
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//lignes_restantes_a_analyser.csv"

# Charger le fichier
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Colonnes d'écart pour les dépenses externalisées
colonnes_ext = ['L17_ECART', 'L18_ECART', 'L19_ECART', 'L20_ECART', 'L21_ECART']

# Extraire les lignes qui répondent à tous les critères
mask_correspondance = df['CORRESPONDANCE_CIR'] == "Non"
mask_cir = df['CIR_RECHERCHE_ECART'] != 0
mask_ext = df[colonnes_ext].abs().sum(axis=1) > 0

# Combiner tous les filtres
df_resultat = df[mask_correspondance & mask_cir & mask_ext]

print(f"Nombre de lignes extraites: {len(df_resultat)}")

# Exporter vers un fichier CSV
output_file = "lignes_ecart_cir_recherche_externalise.csv"
df_resultat.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Lignes exportées dans: {output_file}")

Fichier chargé: 1490 lignes
Nombre de lignes extraites: 379
Lignes exportées dans: lignes_ecart_cir_recherche_externalise.csv
Lignes exportées dans: lignes_ecart_cir_recherche_externalise.csv


In [42]:
import pandas as pd
import numpy as np

# Charger le fichier CSV généré précédemment
fichier = "lignes_ecart_cir_recherche_externalise.csv"
df = pd.read_csv(fichier, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Convertir les colonnes en numérique pour s'assurer que les calculs seront corrects
df['L26A_ECART'] = pd.to_numeric(df['L26A_ECART'], errors='coerce')
df['CIR_TOTAL_ECART'] = pd.to_numeric(df['CIR_TOTAL_ECART'], errors='coerce')

# Créer une colonne pour vérifier si L26A_ECART = CIR_TOTAL_ECART / 0.3
# Permettre une petite marge d'erreur pour les arrondis (0.1%)
df['RATIO'] = df['L26A_ECART'] / (df['CIR_TOTAL_ECART'] / 0.3)
df['EST_EGAL'] = np.isclose(df['RATIO'], 1, rtol=1)  # Tolérance relative de 0.1%

# Compter combien de lignes respectent cette relation
nb_egal = df['EST_EGAL'].sum()
pourcentage = (nb_egal / len(df)) * 100

print(f"\nRésultats de l'analyse:")
print(f"- Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: {nb_egal} sur {len(df)}")
print(f"- Pourcentage: {pourcentage:.2f}%")

# Afficher quelques exemples où la relation est respectée
print("\nExemples où L26A_ECART = CIR_TOTAL_ECART / 3:")
exemples_egal = df[df['EST_EGAL']].head(5)
for _, row in exemples_egal.iterrows():
    print(f"SIREN: {row['SIREN_DECLARANT']}, L26A_ECART: {row['L26A_ECART']}, CIR_TOTAL_ECART/3: {row['CIR_TOTAL_ECART']/3}")
    # Filtrer uniquement les lignes où L26A_ECART = CIR_TOTAL_ECART / 3 est vrai
    df_resultat = df[df['EST_EGAL']]

    # Afficher le nombre de lignes correspondantes
    print(f"Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: {len(df_resultat)}")

    # Exporter les résultats filtrés dans un fichier CSV
    df_resultat.to_csv("lignes_L26A_egal_CIR_TOTAL_div_3.csv", sep=';', encoding='utf-8-sig', index=False)
    # Calculer les sommes
    somme_declare = df_resultat['CIR_TOTAL_DECLARE'].sum()
    somme_calcule = df_resultat['CIR_TOTAL_CALCULE'].sum()

    print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
    print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
    print(f"Résultats exportés dans: lignes_L26A_egal_CIR_TOTAL_div_3.csv")
# Afficher quelques exemples où la relation n'est pas respectée
print("\nExemples où L26A_ECART ≠ CIR_TOTAL_ECART / 3:")
exemples_different = df[~df['EST_EGAL']].head(5)
for _, row in exemples_different.iterrows():
    print(f"SIREN: {row['SIREN_DECLARANT']}, L26A_ECART: {row['L26A_ECART']}, CIR_TOTAL_ECART/3: {row['CIR_TOTAL_ECART']/3}, Ratio: {row['RATIO']:.4f}")

# Exporter les résultats avec la colonne d'analyse supplémentaire
df.to_csv("analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig', index=False)
print(f"\nRésultats détaillés exportés dans: analyse_L26A_vs_CIR_TOTAL.csv")

Fichier chargé: 379 lignes

Résultats de l'analyse:
- Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: 365 sur 379
- Pourcentage: 96.31%

Exemples où L26A_ECART = CIR_TOTAL_ECART / 3:
SIREN: 7080757, L26A_ECART: -5278.76, CIR_TOTAL_ECART/3: -527.8766666666667
Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: 365
Somme CIR_TOTAL_DECLARE : 310,677,342.00 €
Somme CIR_TOTAL_CALCULE : 262,878,908.93 €
Résultats exportés dans: lignes_L26A_egal_CIR_TOTAL_div_3.csv
SIREN: 46750014, L26A_ECART: 205970.78, CIR_TOTAL_ECART/3: 20597.07666666666
Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: 365
Somme CIR_TOTAL_DECLARE : 310,677,342.00 €
Somme CIR_TOTAL_CALCULE : 262,878,908.93 €
Résultats exportés dans: lignes_L26A_egal_CIR_TOTAL_div_3.csv
SIREN: 46750014, L26A_ECART: 205970.78, CIR_TOTAL_ECART/3: 20597.07666666666
Nombre de lignes où L26A_ECART = CIR_TOTAL_ECART / 3: 365
Somme CIR_TOTAL_DECLARE : 310,677,342.00 €
Somme CIR_TOTAL_CALCULE : 262,878,908.93 €
Résultats exportés dans: 

In [43]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas analyse_L26A_vs_CIR_TOTAL.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | siren_analyse_l26a

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\3365988388.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 1111
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur doublement sans motif de la créance total

In [44]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer selon les critères :
# - ECART_RELATIF_POURCENT == -50 ou 50
# - CIR_INNOVATION_ECART == 0
# - CIR_RECHERCHE_ECART entre -1 et 1 inclus
mask = (
    df['ECART_RELATIF_POURCENT'].isin([-50.0, 50.0]) &
    (df['CIR_INNOVATION_ECART'] == 0.0) &
    (df['CIR_RECHERCHE_ECART'].between(-1, 1))
)

df_filtered = df[mask]
print(f"Lignes extraites: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_ecart_relatif_50_innovation0_recherche1.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 1111 lignes
Lignes extraites: 181
Somme CIR_TOTAL_DECLARE : 121,956,843.00 €
Somme CIR_TOTAL_CALCULE : 60,978,422.26 €
Résultats exportés dans: lignes_ecart_relatif_50_innovation0_recherche1.csv
Somme CIR_TOTAL_DECLARE : 121,956,843.00 €
Somme CIR_TOTAL_CALCULE : 60,978,422.26 €
Résultats exportés dans: lignes_ecart_relatif_50_innovation0_recherche1.csv


In [45]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas analyse_L26A_vs_CIR_TOTAL.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\2558383557.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 930

Nombre de lignes restantes à analyser: 930
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## erreur diff Cir total compris entre -500 et 500

In [46]:
import pandas as pd

# Charger le fichier des lignes restantes à analyser
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où CIR_TOTAL_ECART est entre -500 et 500 et CORRESPONDANCE_CIR == "Non"
mask_ecart = df['CIR_TOTAL_ECART'].between(-500, 500) & df['CORRESPONDANCE_CIR'].eq("Non")
df_resultat = df[mask_ecart]

print(f"Nombre de lignes extraites: {len(df_resultat)}")

# Exporter les résultats
output_file = "lignes_ecart_total_entre_-500_et_500.csv"
df_resultat.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_resultat['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_resultat['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 930 lignes
Nombre de lignes extraites: 240
Somme CIR_TOTAL_DECLARE : 53,889,844.00 €
Somme CIR_TOTAL_CALCULE : 53,895,084.56 €
Résultats exportés dans: lignes_ecart_total_entre_-500_et_500.csv
Somme CIR_TOTAL_DECLARE : 53,889,844.00 €
Somme CIR_TOTAL_CALCULE : 53,895,084.56 €
Résultats exportés dans: lignes_ecart_total_entre_-500_et_500.csv


In [47]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas lignes_ecart_total_entre_-500_et_500.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | siren_ecart_total500

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\1236064935.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 690

Nombre de lignes restantes à analyser: 690
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Plafond Partout

In [48]:
import pandas as pd

# Charger le fichier avec les 703 lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où ECART_RELATIF_POURCENT est égal à -100
df_ecart_negatif = df[df['ECART_RELATIF_POURCENT'] == -100]
print(f"Nombre de lignes avec ECART_RELATIF_POURCENT = -100: {len(df_ecart_negatif)}")

# Exporter les résultats dans un fichier CSV
output_file = "lignes_ecart_relatif_moins_100.csv"
df_ecart_negatif.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_ecart_negatif['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_ecart_negatif['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 690 lignes
Nombre de lignes avec ECART_RELATIF_POURCENT = -100: 45
Somme CIR_TOTAL_DECLARE : 1,130,244,774.00 €
Somme CIR_TOTAL_CALCULE : 0.00 €
Résultats exportés dans: lignes_ecart_relatif_moins_100.csv


In [49]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas lignes_ecart_relatif_moins_100.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\2882612865.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 645

Nombre de lignes restantes à analyser: 645
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir innovation

In [50]:
import pandas as pd
import numpy as np

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1)
mask = np.isclose(df['CIR_TOTAL_ECART'], df['CIR_INNOVATION_ECART'], atol=1.0)
df_filtered = df[mask]

print(f"\nNombre de lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1): {len(df_filtered)}")

# Afficher quelques exemples
if len(df_filtered) > 0:
    print("\nExemples :")
    for _, row in df_filtered.head().iterrows():
        print(f"SIREN: {row['SIREN_DECLARANT']}")
        print(f"CIR_TOTAL_ECART: {row['CIR_TOTAL_ECART']:.2f}")
        print(f"CIR_INNOVATION_ECART: {row['CIR_INNOVATION_ECART']:.2f}")

# Exporter les résultats
output_file = "lignes_ecart_total_egal_innovation.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"\nRésultats exportés dans: {output_file}")

Fichier chargé: 645 lignes

Nombre de lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1): 245

Exemples :
SIREN: 71802698
CIR_TOTAL_ECART: 29651.05
CIR_INNOVATION_ECART: 29651.05
SIREN: 318291564
CIR_TOTAL_ECART: -2191.06
CIR_INNOVATION_ECART: -2191.06
SIREN: 323378851
CIR_TOTAL_ECART: 1174.65
CIR_INNOVATION_ECART: 1174.65
SIREN: 323540641
CIR_TOTAL_ECART: 1906.06
CIR_INNOVATION_ECART: 1906.00
SIREN: 331099499
CIR_TOTAL_ECART: -1253.73
CIR_INNOVATION_ECART: -1253.73
Somme CIR_TOTAL_DECLARE : 18,696,070.00 €
Somme CIR_TOTAL_CALCULE : 16,240,495.23 €

Résultats exportés dans: lignes_ecart_total_egal_innovation.csv
Somme CIR_TOTAL_DECLARE : 18,696,070.00 €
Somme CIR_TOTAL_CALCULE : 16,240,495.23 €

Résultats exportés dans: lignes_ecart_total_egal_innovation.csv


In [51]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas innovation_egal_total)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\987456505.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 400

Nombre de lignes restantes à analyser: 400
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir Collection

In [52]:
import pandas as pd
import numpy as np

# Charger le fichier
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Créer le masque pour identifier où CIR_TOTAL_ECART = CIR_COLLECTION_ECART (±1)
mask = np.isclose(df['CIR_TOTAL_ECART'], df['CIR_COLLECTION_ECART'], atol=1.0)

# Filtrer les lignes
df_filtered = df[mask]

print(f"\nNombre de lignes où CIR_TOTAL_ECART = CIR_COLLECTION_ECART (±1): {len(df_filtered)}")

# Afficher quelques statistiques sur ces lignes
if len(df_filtered) > 0:
    print("\nQuelques exemples :")
    for _, row in df_filtered.head().iterrows():
        print(f"\nSIREN: {row['SIREN_DECLARANT']}")
        print(f"CIR_TOTAL_ECART: {row['CIR_TOTAL_ECART']:.2f}")
        print(f"CIR_COLLECTION_ECART: {row['CIR_COLLECTION_ECART']:.2f}")

# Exporter les résultats
output_file = "lignes_ecart_total_egal_collection.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"\nRésultats exportés dans: {output_file}")

Fichier chargé: 400 lignes

Nombre de lignes où CIR_TOTAL_ECART = CIR_COLLECTION_ECART (±1): 33

Quelques exemples :

SIREN: 300347085
CIR_TOTAL_ECART: 200000.00
CIR_COLLECTION_ECART: 200000.00

SIREN: 301857074
CIR_TOTAL_ECART: 200000.00
CIR_COLLECTION_ECART: 200000.00

SIREN: 302306626
CIR_TOTAL_ECART: 200000.00
CIR_COLLECTION_ECART: 200000.00

SIREN: 333401271
CIR_TOTAL_ECART: -7289.81
CIR_COLLECTION_ECART: -7290.00

SIREN: 349760918
CIR_TOTAL_ECART: 199999.22
CIR_COLLECTION_ECART: 200000.00
Somme CIR_TOTAL_DECLARE : 7,412,926.00 €
Somme CIR_TOTAL_CALCULE : 10,939,737.25 €

Résultats exportés dans: lignes_ecart_total_egal_collection.csv


In [53]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas collection_egal_total)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\4065925440.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 367

Nombre de lignes restantes à analyser: 367
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir Déclarer vaut 0 alors que sa ne l'est pas

In [54]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où ECART_RELATIF_POURCENT = 100 et CIR_TOTAL_DECLARE = 0
df_filtered = df[(df['ECART_RELATIF_POURCENT'] == 100) & (df['CIR_TOTAL_DECLARE'] == 0)]

print(f"Nombre de lignes où ECART_RELATIF_POURCENT = 100 et CIR_TOTAL_DECLARE = 0: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_ecart_100_cir_declare_0.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 367 lignes
Nombre de lignes où ECART_RELATIF_POURCENT = 100 et CIR_TOTAL_DECLARE = 0: 44
Somme CIR_TOTAL_DECLARE : 0.00 €
Somme CIR_TOTAL_CALCULE : 2,205,497.50 €
Résultats exportés dans: lignes_ecart_100_cir_declare_0.csv


In [55]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas ecart_100_cir_declare_0)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\1938235922.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 323

Nombre de lignes restantes à analyser: 323
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur liée à L6

In [56]:
import pandas as pd
import numpy as np

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où CIR_TOTAL_ECART = CIR_RECHERCHE_ECART (±1)
mask_ecart = np.isclose(df['CIR_TOTAL_ECART'], df['CIR_RECHERCHE_ECART'], atol=1.0)

# Filtrer les lignes où L6_ECART est strictement en dehors de [-1, 1]
mask_l6 = (df['L6_ECART'] < -1) | (df['L6_ECART'] > 1)

# Appliquer les deux filtres
df_filtered = df[mask_ecart & mask_l6]

print(f"\nNombre de lignes où CIR_TOTAL_ECART = CIR_RECHERCHE_ECART (±1) et L6_ECART hors [-1, 1]: {len(df_filtered)}")
# Exporter les résultats
output_file = "lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"\nRésultats exportés dans: {output_file}")

Fichier chargé: 323 lignes

Nombre de lignes où CIR_TOTAL_ECART = CIR_RECHERCHE_ECART (±1) et L6_ECART hors [-1, 1]: 35
Somme CIR_TOTAL_DECLARE : 8,632,110.00 €
Somme CIR_TOTAL_CALCULE : 9,282,543.16 €

Résultats exportés dans: lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv


In [57]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')
l6_ecart_hors_1 = pd.read_csv("lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])
siren_l6_ecart_hors_1 = set(l6_ecart_hors_1['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas l6_ecart_hors_1)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0 | siren_l6_ecart_hors_1

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\1009379534.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 288

Nombre de lignes restantes à analyser: 288
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur liée à L26

In [58]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Appliquer les filtres demandés
mask = (
    ((df['L26A_ECART'] > 1) | (df['L26A_ECART'] < -1))
    & ((df['L14_ECART'] > -1) & (df['L14_ECART'] < 1))
    & ((df['L20_ECART'] > -1) & (df['L20_ECART'] < 1))
    & ((df['L21_ECART'] > -1) & (df['L21_ECART'] < 1))
    & ((df['L22_ECART'] > -1) & (df['L22_ECART'] < 1))
)

df_filtered = df[mask]
print(f"Nombre de lignes correspondant au cas demandé : {len(df_filtered)}")

# Afficher quelques exemples
if len(df_filtered) > 0:
    print(df_filtered[['SIREN_DECLARANT', 'L26A_ECART', 'L14_ECART', 'L20_ECART', 'L21_ECART']].head())

# Exporter les résultats
output_file = "lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 288 lignes
Nombre de lignes correspondant au cas demandé : 28
    SIREN_DECLARANT  L26A_ECART  L14_ECART  L20_ECART  L21_ECART
7         327880001  -100000.04      -0.04        0.0        0.0
11        335187696   400000.78       0.78        0.0        0.0
16        343043790   200000.63       0.63        0.0        0.0
31        389019274  -200000.45      -0.45        0.0        0.0
51        425112612   -99999.89       0.11        0.0        0.0
Somme CIR_TOTAL_DECLARE : 19,583,069.00 €
Somme CIR_TOTAL_CALCULE : 20,493,169.63 €
Résultats exportés dans: lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv


In [59]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')
l6_ecart_hors_1 = pd.read_csv("lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv", sep=';', encoding='utf-8-sig')
cas_l26a = pd.read_csv("lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])
siren_l6_ecart_hors_1 = set(l6_ecart_hors_1['SIREN_DECLARANT'])
siren_cas_l26a = set(cas_l26a['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas l6_ecart_hors_1)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0 | siren_l6_ecart_hors_1 | siren_cas_l26a

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\2783479905.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 260

Nombre de lignes restantes à analyser: 260
Résultats exportés dans: lignes_restantes_a_analyser.csv
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur oublie CRC dans Cir Total Déclarer

In [60]:
import pandas as pd
import numpy as np

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où les écarts CIR (recherche, collection, innovation, collaboratif) sont tous entre -1 et 1
mask_ecarts = (
    df['CIR_RECHERCHE_ECART'].between(-1, 1) &
    df['CIR_COLLECTION_ECART'].between(-1, 1) &
    df['CIR_INNOVATION_ECART'].between(-1, 1) &
    df['CIR_COLLABORATIF_ECART'].between(-1, 1)
)

# Filtrer les lignes où CIR_TOTAL_ECART = L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE (à ±1 près)
if 'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE' in df.columns:
    mask_l91 = np.isclose(df['CIR_TOTAL_ECART'], df['L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE'], atol=1.0)
else:
    mask_l91 = pd.Series([False]*len(df), index=df.index)
    print("Colonne L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE absente du fichier.")

# Appliquer les deux filtres
df_filtered = df[mask_ecarts & mask_l91]

print(f"Nombre de lignes correspondant au cas demandé : {len(df_filtered)}")

# Afficher quelques exemples
if len(df_filtered) > 0:
    print(df_filtered[['SIREN_DECLARANT', 'CIR_TOTAL_ECART', 'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE']].head())

# Exporter les résultats
output_file = "lignes_ecarts_cir_entre_-1_1_et_total_egal_L91.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 260 lignes
Nombre de lignes correspondant au cas demandé : 140
    SIREN_DECLARANT  CIR_TOTAL_ECART  \
0         300163896         12824.45   
1         300817954         41012.11   
2         304189590         21231.03   
3         314527557         86116.07   
12        339918492          6405.25   

    L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE  
0                                         12824.0  
1                                         41012.5  
2                                         21230.8  
3                                         86116.0  
12                                         6405.6  
Somme CIR_TOTAL_DECLARE : 61,721,671.00 €
Somme CIR_TOTAL_CALCULE : 67,394,882.63 €
Résultats exportés dans: lignes_ecarts_cir_entre_-1_1_et_total_egal_L91.csv


### mise à jour fichier lignes restante

In [61]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')
l6_ecart_hors_1 = pd.read_csv("lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv", sep=';', encoding='utf-8-sig')
cas_l26a = pd.read_csv("lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv", sep=';', encoding='utf-8-sig')
ecarts_cir_l91 = pd.read_csv("lignes_ecarts_cir_entre_-1_1_et_total_egal_L91.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])
siren_l6_ecart_hors_1 = set(l6_ecart_hors_1['SIREN_DECLARANT'])
siren_cas_l26a = set(cas_l26a['SIREN_DECLARANT'])
siren_ecarts_cir_l91 = set(ecarts_cir_l91['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas l6_ecart_hors_1)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0 | siren_l6_ecart_hors_1 | siren_cas_l26a | siren_ecarts_cir_l91

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

C:\Users\msamb\AppData\Local\Temp\ipykernel_11968\3789977644.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')


Fichier principal chargé: 27957 lignes

Nombre de lignes restantes à analyser: 120
Résultats exportés dans: lignes_restantes_a_analyser.csv

Nombre de lignes restantes à analyser: 120
Résultats exportés dans: lignes_restantes_a_analyser.csv


# Correction Creance

In [4]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# ===================================================================
# FONCTIONS UTILITAIRES
# ===================================================================

def format_siren(siren):
    """Formate les SIREN pour avoir exactement 9 caractères"""
    if pd.isna(siren):
        return siren
    siren_str = str(siren).strip()
    # Gère les formats type '123456789.0'
    if '.' in siren_str:
        siren_str = siren_str.split('.')[0]
    return siren_str.zfill(9)

def convertir_en_nombre(valeur, defaut=0.0):
    """Convertit une valeur en nombre flottant de façon sécurisée"""
    if pd.isna(valeur):
        return defaut
    valeur_str = str(valeur).strip()
    if valeur_str == '':
        return defaut
    try:
        # Remplace la virgule par un point pour la conversion décimale
        return float(valeur_str.replace(',', '.'))
    except ValueError:
        # Retourne la valeur par défaut en cas d'erreur de conversion
        return defaut
    except Exception:
         # Capture d'autres erreurs potentielles
        return defaut

# ===================================================================
# CHARGEMENT ET PRÉPARATION DES DONNÉES
# ===================================================================

def charger_donnees(file_path="Calcul_Creance_CIR.csv"):
    """Charge et prépare les données du fichier CIR"""
    print(f"Chargement et préparation des données depuis: {file_path}...")
    try:
        df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig',
                         converters={'SIREN_DECLARANT': str, 'SIREN_DEPOSANT': str},
                         low_memory=False)
        print(f"Fichier principal chargé: {len(df):,} lignes")
    except FileNotFoundError:
        print(f"ERREUR: Fichier non trouvé: {file_path}.")
        return None
    except Exception as e:
        print(f"ERREUR lors du chargement du fichier principal: {str(e)}")
        return None

    print("Formatage des SIRENs...")
    if 'SIREN_DECLARANT' in df.columns:
        df['SIREN_DECLARANT'] = df['SIREN_DECLARANT'].apply(format_siren)
    else:
        print("ERREUR CRITIQUE: Colonne 'SIREN_DECLARANT' non trouvée.")
        return None
    if 'SIREN_DEPOSANT' in df.columns:
        df['SIREN_DEPOSANT'] = df['SIREN_DEPOSANT'].apply(format_siren)

    if 'TRAITEMENT_APPLIQUE' not in df.columns:
        df['TRAITEMENT_APPLIQUE'] = "Non traité"
    else:
         # Assure que la colonne est de type string
         df['TRAITEMENT_APPLIQUE'] = df['TRAITEMENT_APPLIQUE'].astype(str)

    # Liste exhaustive des colonnes numériques à convertir
    numeric_cols = [
        'L1_DOTATION_AMORT_IMMO', 'L2_DOTATION_AMORT_SINISTR', 'L3_DEPENSES_PERSONNEL_CHERCHEURS',
        'L4_REMUNERATION_INVENTEURS', 'L5_DEPENSES_JEUNES_DOCTEURS', 'L6_AUTRES_DEP_FONCT_CALCULE',
        'L7_TOTAL_DEP_FONCT_DECLARE', 'L7_TOTAL_DEP_FONCT_CALCULE', 'L8_FRAIS_BREVETS_COV',
        'L9_DEPENSES_DEFENSE_BREVETS', 'L10_DOTATION_AMORT_BREVETS', 'L11_DEPENSES_NORMALISATION_DECLARE',
        'L12_PRIMES_COTISATIONS_BRUT', 'L12_PRIMES_COTISATIONS_PLAFONNEES', 'L13_VEILLE_TECHNO_BRUT',
        'L13_VEILLE_TECHNO_PLAFONNEE', 'L14_TOTAL_DEPENSES_INTERNES_CALCULE', 'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE',
        'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE', 'L23A_SUBVENTIONS', 'L23B_SOMMES_ENCAISSEES_TIERS',
        'L24_DEPENSES_CONSEIL_CIR', 'L25_REMBOURSEMENTS_SUBVENTIONS', 'L26A_MONTANT_NET_DEPENSES_CALCULE',
        'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE', 'CIR_RECHERCHE_CALCULE', 'CIR_COLLECTION_CALCULE',
        'CIR_INNOVATION_CALCULE', 'CIR_COLLABORATIF_CALCULE', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE',
        'CIR_TOTAL_ECART', 'ECART_RELATIF_POURCENT', 'CIR_RECHERCHE_ECART', 'CIR_INNOVATION_ECART',
        'CIR_COLLECTION_ECART', 'CIR_COLLABORATIF_ECART', 'L6_ECART', 'L14_ECART', 'L17_ECART', 'L18_ECART',
        'L19_ECART', 'L20_ECART', 'L21_ECART', 'L22_ECART', 'L26A_ECART', 'L26B_ECART',
        'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE', 'CIR_TOTAL_CORRIGE' # Inclure si existe déjà
    ]

    print("Conversion des colonnes numériques...")
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertir_en_nombre)

    # Initialisation de CIR_TOTAL_CORRIGE après les conversions
    if 'CIR_TOTAL_CORRIGE' not in df.columns:
        print("Initialisation de la colonne 'CIR_TOTAL_CORRIGE'...")
        if 'CIR_TOTAL_CALCULE' in df.columns:
            df['CIR_TOTAL_CORRIGE'] = df['CIR_TOTAL_CALCULE'].copy()
        else:
            print("  Attention: 'CIR_TOTAL_CALCULE' non trouvé. 'CIR_TOTAL_CORRIGE' initialisé à 0.0.")
            df['CIR_TOTAL_CORRIGE'] = 0.0
    else:
        # Assurer qu'elle est bien numérique si elle existait
        df['CIR_TOTAL_CORRIGE'] = df['CIR_TOTAL_CORRIGE'].apply(convertir_en_nombre)
        print("La colonne 'CIR_TOTAL_CORRIGE' existait déjà et a été (re)convertie en numérique.")

    # Vérification finale des colonnes essentielles
    essential_cols = ['SIREN_DECLARANT', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_CORRIGE']
    for col in essential_cols:
        if col not in df.columns:
            print(f"ERREUR CRITIQUE: Colonne essentielle '{col}' est manquante après chargement/préparation.")
            return None

    print("Préparation des données terminée.")
    return df

def charger_fichier_intermediaire(file_path, main_df_columns):
    """Charge un fichier intermédiaire, formate SIREN et convertit num."""
    try:
        df_inter = pd.read_csv(file_path, sep=';', encoding='utf-8-sig',
                               converters={'SIREN_DECLARANT': str, 'SIREN_DEPOSANT': str},
                               low_memory=False)
        if 'SIREN_DECLARANT' in df_inter.columns:
            df_inter['SIREN_DECLARANT'] = df_inter['SIREN_DECLARANT'].apply(format_siren)
        else:
             print(f"Attention: 'SIREN_DECLARANT' manquant dans '{file_path}'.")
             return pd.DataFrame(columns=main_df_columns)

        numeric_cols_inter = [
            'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_ECART',
            'CIR_RECHERCHE_ECART', 'CIR_COLLECTION_ECART', 'CIR_INNOVATION_ECART', 'CIR_COLLABORATIF_CALCULE',
             'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE'
        ]
        for col in numeric_cols_inter:
            if col in df_inter.columns:
                df_inter[col] = df_inter[col].apply(convertir_en_nombre)
        if 'EST_EGAL' in df_inter.columns:
             df_inter['EST_EGAL'] = df_inter['EST_EGAL'].apply(lambda x: str(x).strip().lower() == 'true')
        return df_inter
    except FileNotFoundError:
        print(f"Info: Fichier intermédiaire '{file_path}' non trouvé.")
        return pd.DataFrame(columns=main_df_columns)
    except Exception as e:
        print(f"Erreur chargement fichier intermédiaire '{file_path}': {str(e)}")
        return pd.DataFrame(columns=main_df_columns)

# ===================================================================
# FONCTIONS DE CORRECTION (1 à 12 Inchangées, 13 Modifiée, 14 Supprimée, 15 Inchangée)
# ===================================================================

# --- Fonctions 1 à 12 : Code simplifié pour la longueur, mais identique à la version précédente ---
def correction_erreurs_personnel(df, processed_sirens, fichier_intermediaire="lignes_jeunes_docteurs_0_amort_sinistr_non0.csv"):
    """Corrige les erreurs de dépenses de personnel (jeunes docteurs)."""
    print("\n1. CORRECTION ERREURS DÉPENSES PERSONNEL (JEUNES DOCTEURS)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        l1_val=convertir_en_nombre(df.loc[idx,'L1_DOTATION_AMORT_IMMO']); l2_val=convertir_en_nombre(df.loc[idx,'L2_DOTATION_AMORT_SINISTR'])
        l4_val=convertir_en_nombre(df.loc[idx,'L4_REMUNERATION_INVENTEURS']); l5_val=convertir_en_nombre(df.loc[idx,'L5_DEPENSES_JEUNES_DOCTEURS'])
        l8_val=convertir_en_nombre(df.loc[idx,'L8_FRAIS_BREVETS_COV']); l9_val=convertir_en_nombre(df.loc[idx,'L9_DEPENSES_DEFENSE_BREVETS'])
        l10_val=convertir_en_nombre(df.loc[idx,'L10_DOTATION_AMORT_BREVETS']); l11_val=convertir_en_nombre(df.loc[idx,'L11_DEPENSES_NORMALISATION_DECLARE'])
        l12_brut_val=convertir_en_nombre(df.loc[idx,'L12_PRIMES_COTISATIONS_BRUT']); l13_brut_val=convertir_en_nombre(df.loc[idx,'L13_VEILLE_TECHNO_BRUT'])
        l21_calcule_val=convertir_en_nombre(df.loc[idx,'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE']); l23a_val=convertir_en_nombre(df.loc[idx,'L23A_SUBVENTIONS'])
        l23b_val=convertir_en_nombre(df.loc[idx,'L23B_SOMMES_ENCAISSEES_TIERS']); l24_val=convertir_en_nombre(df.loc[idx,'L24_DEPENSES_CONSEIL_CIR'])
        l25_val=convertir_en_nombre(df.loc[idx,'L25_REMBOURSEMENTS_SUBVENTIONS']); cir_coll_val=convertir_en_nombre(df.loc[idx,'CIR_COLLECTION_CALCULE'])
        cir_inno_val=convertir_en_nombre(df.loc[idx,'CIR_INNOVATION_CALCULE']); cir_collab_val=convertir_en_nombre(df.loc[idx,'CIR_COLLABORATIF_CALCULE'])
        l26b_val=convertir_en_nombre(df.loc[idx,'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE'])
        df.loc[idx,'L3_DEPENSES_PERSONNEL_CHERCHEURS']=l2_val; df.loc[idx,'L2_DOTATION_AMORT_SINISTR']=0.0
        dep_cherch_tech_updated=df.loc[idx,'L3_DEPENSES_PERSONNEL_CHERCHEURS']; l2_updated=df.loc[idx,'L2_DOTATION_AMORT_SINISTR']
        autres_dep_fonct=(l1_val*0.75)+((dep_cherch_tech_updated+l4_val)*0.43)+l5_val; df.loc[idx,'L6_AUTRES_DEP_FONCT_CALCULE']=autres_dep_fonct
        total_dep_fonct=l1_val+l2_updated+dep_cherch_tech_updated+l4_val+l5_val+autres_dep_fonct; df.loc[idx,'L7_TOTAL_DEP_FONCT_CALCULE']=total_dep_fonct
        prim_cotiz_plaf=min(l12_brut_val,60000); dep_veil_techno_plaf=min(l13_brut_val,60000)
        total_dep_internes=total_dep_fonct+l8_val+l9_val+l10_val+l11_val+prim_cotiz_plaf+dep_veil_techno_plaf; df.loc[idx,'L14_TOTAL_DEPENSES_INTERNES_CALCULE']=total_dep_internes
        total_dep_recherche=total_dep_internes+l21_calcule_val; df.loc[idx,'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE']=total_dep_recherche
        montant_net=total_dep_recherche-l23a_val-l23b_val-l24_val+l25_val; df.loc[idx,'L26A_MONTANT_NET_DEPENSES_CALCULE']=montant_net
        taux_dom,taux_non_dom=0.50,0.30
        if 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE' in df.columns and l26b_val > 0 :
             montant_net_non_dom = max(0, montant_net - l26b_val); cir_recherche = (montant_net_non_dom * taux_non_dom) + (l26b_val * taux_dom)
        else: cir_recherche = montant_net * taux_non_dom
        cir_recherche=max(0, cir_recherche); df.loc[idx,'CIR_RECHERCHE_CALCULE']=cir_recherche
        cir_total=cir_recherche+cir_coll_val+cir_inno_val+cir_collab_val; df.loc[idx,'CIR_TOTAL_CALCULE']=cir_total
        df.loc[idx,'CIR_TOTAL_CORRIGE']=cir_total; df.loc[idx,'TRAITEMENT_APPLIQUE']="Err_dep_personnel"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count+=1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_erreurs_frais_brevets(df, processed_sirens, fichier_intermediaire="lignes_L7_egal_L8.csv"):
    """Corrige les erreurs où L7 (déclaré) est égal à L8."""
    print("\n2. CORRECTION ERREUR L7 = L8 (FRAIS BREVETS)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        l1_val=convertir_en_nombre(df.loc[idx,'L1_DOTATION_AMORT_IMMO']); l2_val=convertir_en_nombre(df.loc[idx,'L2_DOTATION_AMORT_SINISTR'])
        l3_val=convertir_en_nombre(df.loc[idx,'L3_DEPENSES_PERSONNEL_CHERCHEURS']); l4_val=convertir_en_nombre(df.loc[idx,'L4_REMUNERATION_INVENTEURS'])
        l5_val=convertir_en_nombre(df.loc[idx,'L5_DEPENSES_JEUNES_DOCTEURS']); l9_val=convertir_en_nombre(df.loc[idx,'L9_DEPENSES_DEFENSE_BREVETS'])
        l10_val=convertir_en_nombre(df.loc[idx,'L10_DOTATION_AMORT_BREVETS']); l11_val=convertir_en_nombre(df.loc[idx,'L11_DEPENSES_NORMALISATION_DECLARE'])
        l12_plaf_val=convertir_en_nombre(df.loc[idx,'L12_PRIMES_COTISATIONS_PLAFONNEES']); l13_plaf_val=convertir_en_nombre(df.loc[idx,'L13_VEILLE_TECHNO_PLAFONNEE'])
        l21_calcule_val=convertir_en_nombre(df.loc[idx,'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE']); l23a_val=convertir_en_nombre(df.loc[idx,'L23A_SUBVENTIONS'])
        l23b_val=convertir_en_nombre(df.loc[idx,'L23B_SOMMES_ENCAISSEES_TIERS']); l24_val=convertir_en_nombre(df.loc[idx,'L24_DEPENSES_CONSEIL_CIR'])
        l25_val=convertir_en_nombre(df.loc[idx,'L25_REMBOURSEMENTS_SUBVENTIONS']); cir_coll_val=convertir_en_nombre(df.loc[idx,'CIR_COLLECTION_CALCULE'])
        cir_inno_val=convertir_en_nombre(df.loc[idx,'CIR_INNOVATION_CALCULE']); cir_collab_val=convertir_en_nombre(df.loc[idx,'CIR_COLLABORATIF_CALCULE'])
        l26b_val=convertir_en_nombre(df.loc[idx,'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE'])
        df.loc[idx, 'L8_FRAIS_BREVETS_COV'] = 0.0; l8_corrected_val = 0.0
        autres_dep_fonct = (l1_val*0.75) + ((l3_val + l4_val)*0.43) + l5_val; df.loc[idx, 'L6_AUTRES_DEP_FONCT_CALCULE'] = autres_dep_fonct
        total_dep_fonct = l1_val+l2_val+l3_val+l4_val+l5_val+autres_dep_fonct; df.loc[idx, 'L7_TOTAL_DEP_FONCT_CALCULE'] = total_dep_fonct
        total_dep_internes = total_dep_fonct + l8_corrected_val + l9_val + l10_val + l11_val + l12_plaf_val + l13_plaf_val
        df.loc[idx, 'L14_TOTAL_DEPENSES_INTERNES_CALCULE'] = total_dep_internes
        total_dep_recherche = total_dep_internes + l21_calcule_val; df.loc[idx, 'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE'] = total_dep_recherche
        montant_net = total_dep_recherche - l23a_val - l23b_val - l24_val + l25_val; df.loc[idx, 'L26A_MONTANT_NET_DEPENSES_CALCULE'] = montant_net
        taux_dom, taux_non_dom = 0.50, 0.30
        if 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE' in df.columns and l26b_val > 0 :
             montant_net_non_dom = max(0, montant_net - l26b_val); cir_recherche = (montant_net_non_dom * taux_non_dom) + (l26b_val * taux_dom)
        else: cir_recherche = montant_net * taux_non_dom
        cir_recherche = max(0, cir_recherche); df.loc[idx, 'CIR_RECHERCHE_CALCULE'] = cir_recherche
        cir_total = cir_recherche + cir_coll_val + cir_inno_val + cir_collab_val; df.loc[idx, 'CIR_TOTAL_CALCULE'] = cir_total
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = cir_total; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_Frais_BRV"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_recherche_manquant(df, processed_sirens, fichier_intermediaire="lignes_depenses_non0_cir_recherche_0.csv"):
    """Corrige les cas où le CIR recherche déclaré est manquant ou incohérent."""
    print("\n3. CORRECTION CIR RECHERCHE MANQUANT/INCOHÉRENT")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_CIR_RECH"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_erreurs_depenses_externalisees(df, processed_sirens, fichier_intermediaire="analyse_L26A_vs_CIR_TOTAL.csv"):
    """Corrige les erreurs liées aux dépenses externalisées (basé sur L26A vs CIR Total)."""
    print("\n4. CORRECTION ERREURS DÉPENSES EXTERNALISÉES (L26A vs CIR)")
    df_error_raw = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error_raw.empty: return 0
    df_error = df_error_raw
    if 'EST_EGAL' in df_error.columns: df_error = df_error_raw[df_error_raw['EST_EGAL']].copy()
    else: print(f"Attention: Colonne 'EST_EGAL' non trouvée dans '{fichier_intermediaire}'.")
    if df_error.empty: print(f"Aucune donnée pertinente (EST_EGAL=True) trouvée. Skipping."); return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_dep_ext_plaf"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues (EST_EGAL=True), {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_doublement_sans_motif(df, processed_sirens, fichier_intermediaire="lignes_ecart_relatif_50_innovation0_recherche1.csv"):
    """Corrige les erreurs de doublement apparent (écart relatif ~50%). Prend la moyenne."""
    print("\n5. CORRECTION DOUBLEMENT SANS MOTIF (ÉCART ~50%)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        declared = df.loc[idx, 'CIR_TOTAL_DECLARE']; calculated = df.loc[idx, 'CIR_TOTAL_CALCULE']
        corrected = calculated if abs(declared) < 0.01 else (declared + calculated) / 2
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = corrected; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_dbl_creance_totale"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_ecart_petit(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_entre_-500_et_500.csv"):
    """Corrige les petits écarts (-500 à 500 EUR) en prenant la moyenne."""
    print("\n6. CORRECTION PETIT ÉCART (-500 à 500 EUR)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        declared = df.loc[idx, 'CIR_TOTAL_DECLARE']; calculated = df.loc[idx, 'CIR_TOTAL_CALCULE']
        corrected = (declared + calculated) / 2
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = corrected; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_petite"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_plafond_partout(df, processed_sirens, fichier_intermediaire="lignes_ecart_relatif_moins_100.csv"):
    """Corrige les cas où l'écart est -100% (calculé=0), utilise la valeur calculée (0)."""
    print("\n7. CORRECTION PLAFOND PARTOUT (ÉCART -100%)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE'] # Should be ~0
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_PLAF_EVW"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_innovation(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_egal_innovation.csv"):
    """Corrige les cas où l'écart total correspond au CIR Innovation calculé."""
    print("\n8. CORRECTION CIR INNOVATION MANQUANT (ÉCART = INNOVATION)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "innov_manquant"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_collection(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_egal_collection.csv"):
    """Corrige les cas où l'écart total correspond au CIR Collection calculé."""
    print("\n9. CORRECTION CIR COLLECTION MANQUANT (ÉCART = COLLECTION)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "collect_manquant"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_declare_zero(df, processed_sirens, fichier_intermediaire="lignes_ecart_100_cir_declare_0.csv"):
    """Corrige les cas où le CIR déclaré est 0 mais le calculé est positif."""
    print("\n10. CORRECTION CIR DÉCLARÉ = 0 (ÉCART +100%)")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = 0.0 # Déclaré est 0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "err_cir_total_declarer"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_erreurs_l6(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv"):
    """Corrige les erreurs liées à la ligne L6 (Autres dépenses de fonctionnement)."""
    print("\n11. CORRECTION ERREURS LIÉES À L6")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "err_l6"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_erreurs_l26(df, processed_sirens, fichier_intermediaire="lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv"):
    """Corrige les erreurs liées à la ligne L26 (Montant net des dépenses)."""
    print("\n12. CORRECTION ERREURS LIÉES À L26")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "err_l26"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count


# --- Correction 13: CRC Manquant (Cas spécifique Total=L91) ---
# *** LABEL MODIFIÉ ICI ***
def correction_crc_manquant_cir_total(df, processed_sirens, fichier_intermediaire="lignes_ecarts_cir_entre_-1_1_et_total_egal_L91.csv"):
    """Corrige les erreurs où CRC est manquant dans le CIR total (cas spécifique Total=L91). Label -> err_oubli_crc"""
    print("\n13. CORRECTION OUBLI CRC (Cas spécifique Total=L91)") # Titre mis à jour
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    if 'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE' not in df.columns:
        print("Colonne 'L91...' manquante. Skipping correction 13.")
        return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        declared = df.loc[idx, 'CIR_TOTAL_DECLARE']
        crc = df.loc[idx, 'L91_MONTANT_TOTAL_CIR_RECHERCHE_COLLABORATIVE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = declared + crc
        # ---------- LABEL MODIFIÉ ----------
        df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "err_oubli_crc"
        # ----------------------------------
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    # Mettre à jour le label dans le message final
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} € (Label: err_oubli_crc)")
    return count

# --- Correction 14: SUPPRIMÉE ---
# La fonction correction_crc_manquant_simple n'est plus définie.

# --- Correction 15: Valeur Calculée pour le Reste (Inchangée) ---
def correction_valeur_calculee_reste(df, processed_sirens, fichier_intermediaire="lignes_restantes_a_analyser.csv"):
    """Applique la valeur calculée aux cas restants identifiés."""
    print("\n15. VALEUR CALCULÉE PRIVILÉGIÉE POUR LE RESTE IDENTIFIÉ")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value
        df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Valeur calculée privilégiée(Reste)"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

# ===================================================================
# FONCTION PRINCIPALE (Orchestration - Appel à la fonction 14 supprimé)
# ===================================================================

def corriger_cir(file_path="Calcul_Creance_CIR.csv"):
    """Fonction principale pour charger, corriger et sauvegarder le CIR."""
    df = charger_donnees(file_path)
    if df is None:
        print("\nArrêt du traitement.")
        return None

    original_declared = df['CIR_TOTAL_DECLARE'].sum()
    original_calculated = df['CIR_TOTAL_CALCULE'].sum()
    initial_corrected_sum = df['CIR_TOTAL_CORRIGE'].sum()
    print(f"\nSommes avant corrections spécifiques:")
    print(f"  - CIR Déclaré Total: {original_declared:,.2f} €")
    print(f"  - CIR Calculé Total: {original_calculated:,.2f} €")
    print(f"  - CIR Corrigé Initial Total: {initial_corrected_sum:,.2f} €")

    processed_sirens = set()
    total_corrected_count = 0

    print("\n--- Début de l'application des corrections spécifiques ---")
    # Liste des fonctions de correction et de leurs fichiers associés
    # La fonction 14 a été retirée de cette séquence
    correction_definitions = [
        (correction_erreurs_personnel, "lignes_jeunes_docteurs_0_amort_sinistr_non0.csv"), #1
        (correction_erreurs_frais_brevets, "lignes_L7_egal_L8.csv"), #2
        (correction_cir_recherche_manquant, "lignes_depenses_non0_cir_recherche_0.csv"), #3
        (correction_erreurs_depenses_externalisees, "analyse_L26A_vs_CIR_TOTAL.csv"), #4
        (correction_doublement_sans_motif, "lignes_ecart_relatif_50_innovation0_recherche1.csv"), #5
        (correction_ecart_petit, "lignes_ecart_total_entre_-500_et_500.csv"), #6
        (correction_plafond_partout, "lignes_ecart_relatif_moins_100.csv"), #7
        (correction_cir_innovation, "lignes_ecart_total_egal_innovation.csv"), #8
        (correction_cir_collection, "lignes_ecart_total_egal_collection.csv"), #9
        (correction_cir_declare_zero, "lignes_ecart_100_cir_declare_0.csv"), #10
        (correction_erreurs_l6, "lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv"), #11
        (correction_erreurs_l26, "lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv"), #12
        (correction_crc_manquant_cir_total, "lignes_ecarts_cir_entre_-1_1_et_total_egal_L91.csv"), #13 (Label -> err_oubli_crc)
        # Appel à la fonction 14 SUPPRIMÉ
        (correction_valeur_calculee_reste, "lignes_restantes_a_analyser.csv") #15
    ]

    # Exécution séquentielle des corrections
    for func, file in correction_definitions:
         count = func(df, processed_sirens, file)
         total_corrected_count += count

    print(f"\n--- Fin de l'application des corrections spécifiques ---")
    print(f"Nombre total de modifications appliquées par les fonctions actives: {total_corrected_count:,}")
    print(f"Nombre de SIRENs uniques traités: {len(processed_sirens):,}")

    # --- Ajustement final : Mise à zéro des CIR corrigés négatifs ---
    print("\n--- Ajustement final : Mise à zéro des CIR corrigés négatifs ---")
    if 'CIR_TOTAL_CORRIGE' in df.columns:
        # Assurer le type numérique avant la comparaison
        if not pd.api.types.is_numeric_dtype(df['CIR_TOTAL_CORRIGE']):
             df['CIR_TOTAL_CORRIGE'] = df['CIR_TOTAL_CORRIGE'].apply(convertir_en_nombre)

        negative_mask = df['CIR_TOTAL_CORRIGE'] < 0
        count_negative = negative_mask.sum()
        if count_negative > 0:
            sum_before_zeroing = df.loc[negative_mask, 'CIR_TOTAL_CORRIGE'].sum()
            df.loc[negative_mask, 'CIR_TOTAL_CORRIGE'] = 0.0
            print(f"{count_negative:,} lignes avec CIR_TOTAL_CORRIGE négatif ont été mises à 0 (Impact: {-sum_before_zeroing:,.2f} €).")
        else: print("Aucune valeur négative trouvée dans CIR_TOTAL_CORRIGE après corrections.")
    else: print("Erreur: Colonne 'CIR_TOTAL_CORRIGE' non trouvée pour l'ajustement final.")

    # --- Finalisation des lignes 'Non traité' ---
    mask_non_traite_final = df['TRAITEMENT_APPLIQUE'] == "Non traité"
    count_non_traite = mask_non_traite_final.sum()
    print(f"\nConfirmation pour les lignes restées 'Non traité' ({count_non_traite:,}):")
    if count_non_traite > 0:
        # Assurer numérique et non-négatif
        df.loc[mask_non_traite_final, 'CIR_TOTAL_CORRIGE'] = df.loc[mask_non_traite_final, 'CIR_TOTAL_CORRIGE'].apply(convertir_en_nombre)
        neg_in_non_traite_mask = mask_non_traite_final & (df['CIR_TOTAL_CORRIGE'] < 0)
        count_neg_in_non_traite = neg_in_non_traite_mask.sum()
        if count_neg_in_non_traite > 0:
             sum_neg_non_traite = df.loc[neg_in_non_traite_mask, 'CIR_TOTAL_CORRIGE'].sum()
             df.loc[neg_in_non_traite_mask, 'CIR_TOTAL_CORRIGE'] = 0.0
             print(f"  - {count_neg_in_non_traite} de ces lignes étaient négatives et mises à 0 (impact: {-sum_neg_non_traite:,.2f} €).")
        else: print("  - Aucune valeur négative parmi les lignes 'Non traité'.")
    else: print("  - Aucune ligne marquée comme 'Non traité'.")

    # --- Calcul des totaux finaux ---
    final_total_corrected = df['CIR_TOTAL_CORRIGE'].sum()

    # --- Sauvegarde ---
    output_file = "Calcul_Creance_CIR_Corrige.csv"
    print(f"\nSauvegarde du fichier corrigé sous: {output_file}")
    try:
        df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False, decimal='.')
        print(f"Fichier sauvegardé avec succès.")
    except Exception as e: print(f"ERREUR lors de la sauvegarde: {str(e)}")

    # --- Résumé Final ---
    print("\n=====================================================")
    print("                RÉSUMÉ FINAL")
    print("=====================================================")
    print(f"Montant total CIR déclaré (Original):       {original_declared:,.2f} €")
    print(f"Montant total CIR calculé (Original):       {original_calculated:,.2f} €")
    print(f"Montant total CIR après corrections:        {final_total_corrected:,.2f} €")
    initial_ecart = original_calculated - original_declared
    final_ecart = final_total_corrected - original_declared
    improvement = abs(initial_ecart) - abs(final_ecart)
    print(f"\nÉcart initial (calculé - déclaré):        {initial_ecart:,.2f} €")
    print(f"Écart final (corrigé - déclaré):          {final_ecart:,.2f} €")
    print(f"Réduction de l'écart absolu:              {improvement:,.2f} €")
    if abs(original_declared) > 0.01:
        print(f"  Écart relatif initial:                  {initial_ecart / original_declared * 100:.2f}%")
        print(f"  Écart relatif final:                    {final_ecart / original_declared * 100:.2f}%")
        if abs(initial_ecart) > 0.01: print(f"  Réduction relative de l'écart:          {improvement / abs(initial_ecart) * 100:.2f}%")
    else: print("\nCalculs relatifs non pertinents (CIR déclaré total proche de zéro).")

    print("\n--- Détails par Type de Traitement Appliqué ---")
    if 'TRAITEMENT_APPLIQUE' in df.columns:
        treatment_counts = df['TRAITEMENT_APPLIQUE'].value_counts().sort_index()
        print("\nNombre d'entreprises par type de traitement final:")
        for treatment, count in treatment_counts.items(): print(f"  - {treatment:<40}: {count:10,}")
        print("\nDétails financiers par type de traitement final:")
        treatment_order = treatment_counts.index.tolist()
        for treatment in treatment_order:
            subset = df[df['TRAITEMENT_APPLIQUE'] == treatment]
            if len(subset) == 0: continue
            declared_subset = subset['CIR_TOTAL_DECLARE'].sum()
            corrected_subset = subset['CIR_TOTAL_CORRIGE'].sum()
            ecart_final_subset = corrected_subset - declared_subset
            print(f"\nTraitement: {treatment}")
            print(f"  Nombre d'entreprises:                 {len(subset):<10,}")
            print(f"  CIR déclaré (groupe):                 {declared_subset:>18,.2f} €")
            print(f"  CIR corrigé final (groupe):           {corrected_subset:>18,.2f} €")
            print(f"  Écart final (corrigé-déclaré groupe): {ecart_final_subset:>18,.2f} €")
            if abs(declared_subset) > 0.01:
                impact_relatif = ecart_final_subset / declared_subset * 100
                print(f"  Impact relatif final (groupe):        {impact_relatif:>17.2f}%")
    else: print("\nColonne 'TRAITEMENT_APPLIQUE' non trouvée.")

    print("\n=====================================================")
    print("Traitement terminé.")
    print("=====================================================")
    return df

# ===================================================================
# EXÉCUTION
# ===================================================================
if __name__ == "__main__":
    # Adapter le chemin si nécessaire
    input_filename = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv"
    print(f"*** DÉBUT DU SCRIPT DE CORRECTION CIR ***")
    # Affichage de la date et heure actuelles - Utilisation de la date système
    current_time_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"Date et heure: {current_time_str}")
    print(f"Fichier d'entrée: {input_filename}")
    df_corrige = corriger_cir(input_filename)
    if df_corrige is not None: print("\nScript terminé avec succès.")
    else: print("\nLe script n'a pas pu se terminer correctement.")

*** DÉBUT DU SCRIPT DE CORRECTION CIR ***
Date et heure: 2025-04-29 14:18:00
Fichier d'entrée: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv
Chargement et préparation des données depuis: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR.csv...
Fichier principal chargé: 27,957 lignes
Formatage des SIRENs...
Conversion des colonnes numériques...
Initialisation de la colonne 'CIR_TOTAL_CORRIGE'...
Préparation des données terminée.

Sommes avant corrections spécifiques:
  - CIR Déclaré Total: 8,875,111,740.00 €
  - CIR Calculé Total: 7,586,181,390.35 €
  - CIR Corrigé Initial Total: 7,586,181,390.35 €

--- Début de l'application des corrections spécifiques ---

1. CORRECTION ERREURS DÉPENSES PERSONNEL (JEUNES DOCTEURS)
-> 16 lignes lues, 16 corrigées. Impact local: 575,806.51 €

2. CORRECTION ERREUR L7 = L8 (FRAIS BREVETS)
-> 1 lignes lues, 1 corrigées. Impact local: -0.64 €

3. CORRECTION CIR RECHERCHE MANQUANT/INCOHÉRENT
-> 23 lignes lu

In [2]:
df_init = pd.read_parquet("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//cir_millesime_2022_ss_dbls.parquet")
df_corrige = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_Corrige.csv", 
                         sep=';', encoding='utf-8-sig', decimal='.',low_memory=False)


In [3]:
# 1. Charger le fichier corrigé existant
df_corrige = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_Corrige.csv", 
                         sep=';', encoding='utf-8-sig', low_memory=False)

# 2. Standardiser les SIREN dans les deux DataFrames
df_corrige['SIREN_STD'] = df_corrige['SIREN_DECLARANT'].astype(str).str.strip().str.zfill(9)
df_init['siren_std'] = df_init['siren_declarant'].astype(str).str.strip().str.zfill(9)

# 3. Créer un dictionnaire de correspondance SIREN → type_final
siren_to_type = dict(zip(df_init['siren_std'], df_init['type_retenu']))

# 4. Appliquer cette correspondance directement
df_corrige['type_retenu'] = df_corrige['SIREN_STD'].map(siren_to_type)

# 5. Supprimer la colonne temporaire SIREN_STD
df_corrige = df_corrige.drop(columns=['SIREN_STD'])

# 6. Sauvegarder le résultat
df_corrige.to_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_Corrige2.csv", 
                  sep=';', encoding='utf-8-sig', index=False, decimal='.')

# 7. Vérifier le résultat
non_null_count = df_corrige['type_retenu'].notna().sum()
print(f"Nombre de lignes avec type_retenu non-null : {non_null_count}")

# 8. Afficher quelques exemples
print("\nExemples de résultats:")
sample = df_corrige[['SIREN_DECLARANT', 'TRAITEMENT_APPLIQUE', 'type_retenu']].sample(5)
print(sample)

Nombre de lignes avec type_retenu non-null : 27957

Exemples de résultats:
       SIREN_DECLARANT TRAITEMENT_APPLIQUE type_retenu
13709        533868873          Non traité         IND
8841         477865562          Non traité       FILLE
23114        842712010          Non traité         IND
1271         327235818          Non traité        MERE
10719        499732212    Err_dep_ext_plaf         IND


In [11]:
# 1. Charger le fichier corrigé existant
df_corrige = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_Corrige3.csv", 
                         sep=';', encoding='utf-8-sig', low_memory=False)

# 2. Standardiser et nettoyer les SIREN
df_corrige['SIREN_DECLARANT_STD'] = df_corrige['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)
df_corrige['SIREN_DEPOSANT_STD'] = df_corrige['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)

# 3. Créer les ensembles de SIREN déclarants et déposants
set_declarants = set(df_corrige['SIREN_DECLARANT_STD'])
set_deposants = set(df_corrige['SIREN_DEPOSANT_STD'])

# 4. Identifier les SIREN qui sont déposants mais pas déclarants (en filtrant les SIREN non valides)
#if siren and not (siren.strip('0') == '')
deposants_non_declarants = {siren for siren in (set_deposants - set_declarants) }
print(f"Nombre de SIREN qui sont déposants mais pas déclarants: {len(deposants_non_declarants)}")

# 5. Créer les nouvelles lignes pour les déposants manquants
new_rows = []

for siren in deposants_non_declarants:
    # Créer un dictionnaire pour la nouvelle ligne
    new_row = {}
    
    # Copier la structure d'une ligne existante pour obtenir toutes les colonnes
    for col in df_corrige.columns:
        if col in ['SIREN_DECLARANT_STD', 'SIREN_DEPOSANT_STD']:
            continue  # On n'inclut pas ces colonnes temporaires
        
        # Déterminer le type et la valeur appropriée
        if col == 'SIREN_DECLARANT' or col == 'SIREN_DEPOSANT':
            # Utiliser toujours le format chaîne pour éviter les problèmes
            new_row[col] = siren
        elif col == 'TRAITEMENT_APPLIQUE':
            new_row[col] = 'Déposant ajouté'
        elif col == 'type_retenu':  # Ajouter "MERE" dans la colonne type_retenu
            new_row[col] = 'MERE'
        elif col == 'DESIGNATION' or col == 'COMPLEMENT_DESIGNATION':
            new_row[col] = '' # Champs texte vide
        elif pd.api.types.is_numeric_dtype(df_corrige[col]):
            # Pour les colonnes numériques, mettre 0
            new_row[col] = 0
        else:
            # Pour les autres colonnes non numériques, mettre une valeur vide
            new_row[col] = ''
    
    # Ajouter la ligne au tableau de nouvelles lignes
    new_rows.append(new_row)

# 6. Créer un DataFrame avec les nouvelles lignes
if new_rows:
    df_new_rows = pd.DataFrame(new_rows)
    
    # 7. Concaténer avec le DataFrame original
    df_corrige_complet = pd.concat([df_corrige.drop(['SIREN_DECLARANT_STD', 'SIREN_DEPOSANT_STD'], axis=1),
                                df_new_rows],
                               ignore_index=True)
    
    # 8. Afficher des informations sur le résultat
    print(f"DataFrame original: {len(df_corrige)} lignes")
    print(f"Nouvelles lignes ajoutées: {len(new_rows)} lignes")
    print(f"DataFrame final: {len(df_corrige_complet)} lignes")
    
    # 9. Sauvegarder le résultat
    output_file = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_Corrige_avec_deposants.csv"
    df_corrige_complet.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
    print(f"\nFichier sauvegardé: {output_file}")
    
    # 10. Afficher un exemple des nouvelles lignes
    print("\nExemple d'une nouvelle ligne ajoutée:")
    if not df_new_rows.empty:
        cols_to_show = ['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'TRAITEMENT_APPLIQUE', 'type_retenu', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_CORRIGE']
        print(df_new_rows[cols_to_show].iloc[0])
else:
    print("Aucun déposant non déclarant trouvé, aucune ligne à ajouter.")

Nombre de SIREN qui sont déposants mais pas déclarants: 2444
DataFrame original: 27957 lignes
Nouvelles lignes ajoutées: 2444 lignes
DataFrame final: 30401 lignes

Fichier sauvegardé: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_Corrige_avec_deposants.csv

Exemple d'une nouvelle ligne ajoutée:
SIREN_DECLARANT              908968548
SIREN_DEPOSANT               908968548
TRAITEMENT_APPLIQUE    Déposant ajouté
type_retenu                       MERE
CIR_TOTAL_DECLARE                    0
CIR_TOTAL_CALCULE                    0
CIR_TOTAL_CORRIGE                    0
Name: 0, dtype: object


## Difference pour verifier si tous les deposants sont declarants.

In [12]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_Corrige_avec_deposants.csv", 
                 sep=';', encoding='utf-8-sig', low_memory=False)

# Standardiser les SIREN (en string, 9 caractères, sans espaces)
set_declarants = set(df['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9))
set_deposants = set(df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9))

# Différence symétrique (éléments dans l'un ou l'autre ensemble, mais pas les deux)
difference_symetrique = set_declarants ^ set_deposants

print(f"Nombre de SIREN présents soit comme déclarant, soit comme déposant (mais pas les deux) : {len(difference_symetrique)}")

if len(difference_symetrique) == 0:
    print("La différence symétrique est vide.")
else:
    print("Exemples de SIREN présents dans un seul ensemble :", list(difference_symetrique)[:5])
    
    # Si vous voulez aussi voir spécifiquement les SIREN qui sont uniquement déclarants
    seulement_declarants = set_declarants - set_deposants
    print(f"\nNombre de SIREN qui sont uniquement déclarants : {len(seulement_declarants)}")
    if len(seulement_declarants) > 0:
        print("Exemples:", list(seulement_declarants)[:5])
    
    # Et les SIREN qui sont uniquement déposants
    seulement_deposants = set_deposants - set_declarants
    print(f"Nombre de SIREN qui sont uniquement déposants : {len(seulement_deposants)}")
    if len(seulement_deposants) > 0:
        print("Exemples:", list(seulement_deposants)[:5])

Nombre de SIREN présents soit comme déclarant, soit comme déposant (mais pas les deux) : 6533
Exemples de SIREN présents dans un seul ensemble : ['775656325', '434039921', '385115142', '378888085', '318365228']

Nombre de SIREN qui sont uniquement déclarants : 6533
Exemples: ['775656325', '422505347', '434039921', '904479672', '385115142']
Nombre de SIREN qui sont uniquement déposants : 0


In [13]:
import pandas as pd
import numpy as np

# 1. Charger le fichier
print("Chargement du fichier...")
df = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_Corrige_avec_deposants.csv", 
                 sep=';', encoding='utf-8-sig', low_memory=False)

# 2. Standardiser les SIREN des déposants pour éviter les problèmes de format
print("Standardisation des SIREN...")
df['SIREN_DEPOSANT_STD'] = df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)

# 3. Calculer la somme du CIR par déposant
print("Calcul de la somme du CIR par déposant...")
# Utiliser CIR_TOTAL_CORRIGE qui est le montant final retenu après corrections
cir_par_deposant = df.groupby('SIREN_DEPOSANT_STD')['CIR_TOTAL_CORRIGE'].sum().reset_index()
cir_par_deposant.rename(columns={'CIR_TOTAL_CORRIGE': 'cir_benef_total_temp'}, inplace=True)

# 4. Fusionner cette somme avec le DataFrame original
print("Ajout de la nouvelle colonne au DataFrame...")
df = df.merge(cir_par_deposant, on='SIREN_DEPOSANT_STD', how='left')

# 5. Mettre à zéro la colonne cir_benef_total pour toutes les lignes qui ne sont pas de type MERE
print("Attribution des valeurs selon le type...")
# Créer la colonne finale 'cir_benef_total'
df['cir_benef_total'] = 0.0  # Initialiser à zéro pour toutes les lignes

# Mettre la somme calculée uniquement pour les lignes de type 'MERE' et 'IND'
df.loc[df['type_retenu'] == 'MERE', 'cir_benef_total'] = df.loc[df['type_retenu'] == 'MERE', 'cir_benef_total_temp']
df.loc[df['type_retenu'] == 'IND', 'cir_benef_total'] = df.loc[df['type_retenu'] == 'IND', 'cir_benef_total_temp']

# Supprimer la colonne temporaire
df.drop(['SIREN_DEPOSANT_STD', 'cir_benef_total_temp'], axis=1, inplace=True)

# 6. Afficher quelques statistiques
print(f"\nNombre total d'entreprises: {len(df)}")
print(f"Nombre de déposants uniques: {df['SIREN_DEPOSANT'].nunique()}")

# Vérifier la distribution des valeurs cir_benef_total selon le type
print("\nDistribution des valeurs cir_benef_total par type:")
print(df.groupby('type_retenu')['cir_benef_total'].describe())

# Trier par montant de cir_benef_total pour voir les déposants avec le plus grand total
top_deposants = df[df['cir_benef_total'] > 0][['SIREN_DEPOSANT', 'cir_benef_total']].drop_duplicates().sort_values('cir_benef_total', ascending=False)
print("\nTop 5 des déposants (MERE) avec le plus grand total de CIR:")
print(top_deposants.head(5))

# 7. Sauvegarder le résultat
output_file = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total.csv"
df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"\nFichier sauvegardé: {output_file}")

# 8. Vérification - calcul de la somme totale de CIR
total_cir_corrige = df['CIR_TOTAL_CORRIGE'].sum()
total_cir_benef = df['cir_benef_total'].sum()
print(f"\nSomme totale de CIR_TOTAL_CORRIGE: {total_cir_corrige:,.2f} €")
print(f"Somme totale de cir_benef_total (pour les MERE et IND): {total_cir_benef:,.2f} €")

Chargement du fichier...
Standardisation des SIREN...
Calcul de la somme du CIR par déposant...
Ajout de la nouvelle colonne au DataFrame...
Attribution des valeurs selon le type...

Nombre total d'entreprises: 30401
Nombre de déposants uniques: 24259

Distribution des valeurs cir_benef_total par type:
               count          mean           std  min         25%         50%  \
type_retenu                                                                     
FILLE         6485.0  0.000000e+00  0.000000e+00  0.0      0.0000       0.000   
IND          19548.0  1.486049e+05  6.405565e+05  0.0  20689.2150   50638.775   
MERE          4368.0  1.098945e+06  5.791363e+06  0.0  45006.6725  120684.815   

                     75%           max  
type_retenu                             
FILLE             0.0000  0.000000e+00  
IND          110767.6575  3.046598e+07  
MERE         415645.3500  1.411021e+08  

Top 5 des déposants (MERE) avec le plus grand total de CIR:
      SIREN_DEPOSANT  ci

In [18]:
import pandas as pd
import numpy as np

# Charger le fichier
df = pd.read_csv(output_file, sep=';', encoding='utf-8-sig', low_memory=False)

# Standardiser les SIREN des déposants
df['SIREN_DEPOSANT_STD'] = df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)

# Calculer la somme du CIR par déposant
cir_par_deposant = df.groupby('SIREN_DEPOSANT_STD')['CIR_TOTAL_CORRIGE'].sum().reset_index()
"""if cir_par_deposant == df['CIR_TOTAL_CORRIGE'].sum():
    print("La somme des CIR par déposant est égale à la somme totale des CIR.")
else:   
    print("La somme des CIR par déposant n'est pas égale à la somme totale des CIR. Vérifiez les données.")"""

cir_par_deposant.rename(columns={'CIR_TOTAL_CORRIGE': 'cir_benef_total_temp'}, inplace=True)

# Fusionner cette somme avec le DataFrame original
df = df.merge(cir_par_deposant, on='SIREN_DEPOSANT_STD', how='left')

# Mettre à zéro la colonne cir_benef_total partout
df['cir_benef_total'] = 0.0

# Mettre la somme uniquement si SIREN_DECLARANT == SIREN_DEPOSANT
mask = df['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9) == df['SIREN_DEPOSANT_STD']
df.loc[mask, 'cir_benef_total'] = df.loc[mask, 'cir_benef_total_temp']

# Supprimer la colonne temporaire
df.drop(['SIREN_DEPOSANT_STD', 'cir_benef_total_temp'], axis=1, inplace=True)

# Sauvegarder le résultat
output_file_final = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final.csv"
df.to_csv(output_file_final, sep=';', encoding='utf-8-sig', index=False)
cir_par_deposant.to_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_par_deposant.csv", sep=';', encoding='utf-8-sig', index=False)
print(f"Fichier sauvegardé: {output_file_final}")

# Vérification
total_cir_corrige = df['CIR_TOTAL_CORRIGE'].sum()
print(f"\nSomme totale de CIR_TOTAL_CORRIGE: {total_cir_corrige:,.2f} €")
print(f"Somme totale de cir_benef_total (cas SIREN_DECLARANT == SIREN_DEPOSANT): {df['cir_benef_total'].sum():,.2f} €")
print(f"difference entre les deux sommes: {total_cir_corrige - df['cir_benef_total'].sum():,.2f} €")

Fichier sauvegardé: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final.csv

Somme totale de CIR_TOTAL_CORRIGE: 7,616,570,450.05 €
Somme totale de cir_benef_total (cas SIREN_DECLARANT == SIREN_DEPOSANT): 7,612,257,192.77 €
difference entre les deux sommes: 4,313,257.28 €


In [16]:
# Identifiez les lignes qui contribuent à la différence
df_diff = df[(df['CIR_TOTAL_CORRIGE'] > 0) & (df['cir_benef_total'] == 0)]

# Vérifiez leurs caractéristiques
print(f"Nombre d'entreprises contribuant à la différence: {len(df_diff)}")
print(df_diff['type_retenu'].value_counts())

# Somme des CIR_TOTAL_CORRIGE pour ces lignes
diff_sum = df_diff['CIR_TOTAL_CORRIGE'].sum()
print(f"Somme des CIR non attribués: {diff_sum:,.2f} €")

Nombre d'entreprises contribuant à la différence: 6484
type_retenu
FILLE    6420
MERE       63
IND         1
Name: count, dtype: int64
Somme des CIR non attribués: 3,724,198,531.98 €
